In [ ]:
# dates in correct format for yahoo finance.
today = date.today() 
today = datetime(today.year,today.month,today.day) #today
one = datetime(today.year-1,today.month,today.day) #one year ago
three = datetime(today.year-3,today.month,today.day) #three years ago
five = datetime(today.year-5,today.month,today.day) #five years ago
ten = datetime(today.year-10,today.month,today.day) #ten years ago
twenty = datetime(today.year-20,today.month,today.day) #twenty years ago


from statsmodels.tsa.stattools import adfuller # skal bruges til stationarity
from scipy.stats import norm # skal bruges til VaR og CVaR

#test for stationarity
def station1(timeseries):
    rolmean = pd.Series.rolling(timeseries,window=int(len(timeseries)/2)).mean()
    rolstd = pd.Series.rolling(timeseries, window=int(len(timeseries)/2)).std()
    #fig = plt.figure()
    #fig.add_subplot()
    #orig = plt.plot(timeseries, color = 'blue',label='Original')
    #mean = plt.plot(rolmean , color = 'red',label = 'rolling mean')
    #std = plt.plot(rolstd, color = 'black', label= 'Rolling standard deviation')
    #plt.ylim([min(timeseries)-3*abs(np.mean(timeseries)),max(timeseries)+3*abs(np.mean(timeseries))])
    #plt.legend(loc = 'best')
    #plt.title('Rolling Mean & Standard Deviation')
    #plt.show(block=False)
    #print( 'Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries,autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4],index = ['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical value (%s)' %key] = value
    #print(dfoutput)
    if dfoutput['p-value'] > 0.05:
        res = "Non-stationary"
    else:
        res = "Stationary"
        
    if dfoutput['Test Statistic'] > dfoutput['Critical value (5%)']:
        res = "Non-stationary"
    else:
        res = "Stationary"
    return res

def station2(x):
    result = adfuller(x)
    #print('ADF Statistic: %f' % result[0])
    #print('p-value: %f' % result[1])
    #print('Critical Values:')
    #for key, value in result[4].items():
        #print('\t%s: %.3f' % (key, value))
    res = "Stationary"
    p = pd.DataFrame(result[4].items())[1]
    if result[1] > 0.05:
        res = "Non-stationary"
    #for i in range(len(p)):
        #if result[0] > p[i]:
            #print("Not within ", pd.DataFrame(result[4].items())[0][i])
    return res


#VaR og CVaR

def var(data, level=5):
    z = norm.ppf(level/100)
    return -(data.mean() + z*data.std(ddof=0))

def cvar(data, level=5):

    confidence_level = 1-(level/100)  # Set the desired confidence level
    sorted_prices = np.sort(data)
    num_samples = len(sorted_prices)
    cvar_index = int((1 - confidence_level) * num_samples)

    cvar = np.mean(sorted_prices[:cvar_index])

    return(cvar)








class financial_analytics:
    
    def __init__(self):
        print("hi")

class tech:
    
    def __init__(self, ticker):
        ########################################################################################################
        # Download base data for long period, daily-1m and monthly-1h
        ########################################################################################################
        self.st = pd.DataFrame()
        t = yf.Ticker(ticker)
        self.st = t.history(start='2020-04-01', end='2024-04-15')
        self.st.index = pd.to_datetime(self.st.index).tz_localize(None)
        self.st_daily = yf.download(ticker, start='2024-04-10', period='1d', interval="1m")
        self.st_month = yf.download(ticker, start='2024-03-01', period='1d', interval="1h")
        ########################################################################################################
        # extract price differentiation
        ########################################################################################################
        self.returns = self.st['Close'].pct_change().dropna()
        self.returns_daily = self.st_daily['Close'].pct_change().dropna()
        self.returns_month = self.st_month['Close'].pct_change().dropna()
        ########################################################################################################
        # extract price accelerations
        ########################################################################################################
        self.acc = self.returns.pct_change().dropna()
        self.acc_daily = self.returns_daily.pct_change().dropna()
        self.acc_month = self.returns_month.pct_change().dropna()
        ########################################################################################################
        # extract volume differentiations
        ########################################################################################################
        self.volume_ret = self.st['Volume'].pct_change().dropna()
        self.volume_ret = self.volume_ret[self.volume_ret != np.inf]
        self.volume_ret_daily = self.st_daily['Volume'].pct_change().dropna()
        self.volume_ret_daily = self.volume_ret_daily[self.volume_ret_daily != np.inf]
        self.volume_ret_month = self.st_month['Volume'].pct_change().dropna()
        self.volume_ret_month = self.volume_ret_month[self.volume_ret_month != np.inf]
        ########################################################################################################
        # extract volume accelerations
        ########################################################################################################
        self.volume_acc = self.volume_ret.pct_change().dropna()
        self.volume_acc_daily = self.volume_ret_daily.pct_change().dropna()
        self.volume_acc_month = self.volume_ret_month.pct_change().dropna()

        ########################################################################################################
        # Moving averages for the long term time series
        ########################################################################################################
        self.ma3_l = self.ma(self.st['Close'], 3)
        self.ma5_l = self.ma(self.st['Close'], 5)
        self.ma10_l = self.ma(self.st['Close'], 10)
        self.ma21_l = self.ma(self.st['Close'], 21)
        self.ma35_l = self.ma(self.st['Close'], 35)
        self.ma60_l = self.ma(self.st['Close'], 60)
        self.ma120_l = self.ma(self.st['Close'], 120)
        ########################################################################################################
        # Moving averages for the last month time series
        ########################################################################################################
        self.ma3_m = self.ma(self.st_month['Close'], 3)
        self.ma5_m = self.ma(self.st_month['Close'], 5)
        self.ma10_m = self.ma(self.st_month['Close'], 10)
        self.ma21_m = self.ma(self.st_month['Close'], 21)
        self.ma35_m = self.ma(self.st_month['Close'], 35)
        self.ma60_m = self.ma(self.st_month['Close'], 60)
        self.ma120_m = self.ma(self.st_month['Close'], 120)
        ########################################################################################################
        # Moving averages for the last 7 days time series
        ########################################################################################################
        self.ma3_s = self.ma(self.st_daily['Close'], 3)
        self.ma5_s = self.ma(self.st_daily['Close'], 5)
        self.ma10_s = self.ma(self.st_daily['Close'], 10)
        self.ma21_s = self.ma(self.st_daily['Close'], 21)
        self.ma35_s = self.ma(self.st_daily['Close'], 35)
        self.ma60_s = self.ma(self.st_daily['Close'], 60)
        self.ma120_s = self.ma(self.st_daily['Close'], 120)
        
        ########################################################################################################
        # Empirical distribution of 
        ########################################################################################################
        
        # are the means and stds the same? are any of the means above 0 or below 0 and is the current situation good, positive mean and low std?
        self.ed_1m_ret = self.ed(self.returns_month,100)
        self.ed_3m_ret = self.ed(self.returns.tail(90),100)
        self.ed_1y_ret = self.ed(self.returns.tail(200),100)
        self.ed_3y_ret = self.ed(self.returns.tail(780),100)
        self.ed_5y_ret = self.ed(self.returns.tail(1250),100)
        
        # expected price range
        self.ed_3y_price = self.ed(self.st.tail(780)['Close'],100) # we want to know mu and sigma
          
            
        ########################################################################################################
        # Simple Volume indicators, we just want to know if it's increasing regularly
        ######################################################################################################## 
            
        ####
        print("Average volume past 5 years: ", np.mean(self.st['Volume'].tail(1250))) 
        print("Average volume past 3 years: ", np.mean(self.st['Volume'].tail(780)))
        print("Average volume past 1 year: ", np.mean(self.st['Volume'].tail(250)))
        
        print("Average return past 5 years: ", np.mean(self.returns.tail(1250))) 
        print("Average return past 3 years: ", np.mean(self.returns.tail(780)))
        print("Average return past 1 year: ", np.mean(self.returns.tail(250)))
        
        ###
        print("Volume/return increase overall: ",  np.mean(self.volume_ret) * np.mean(self.returns))
        print("Volume/return increase past month: ",  np.mean(self.volume_ret_month) * np.mean(self.returns_month))
        print("Volume/return increase past week: ",  np.mean(self.volume_ret_daily) * np.mean(self.returns_daily))    

        self.indicator(self.returns)
        
class portfolio:
    
    def __init__(self, file):
        
        self.portf = pd.read_excel(open(file, 'rb'))
        self.portf = self.portf['Symbols'].tolist()
        
        self.scores = pd.DataFrame(0.0, index = self.portf, columns = ["financials", "holders", "dividend", "technical", "collective"])
        
        today = date.today() 
        self.today = datetime(today.year,today.month,today.day) #today
        self.one = datetime(today.year-1,today.month,today.day) #one year ago
        self.three = datetime(today.year-3,today.month,today.day) #three years ago
        self.five = datetime(today.year-5,today.month,today.day) #five years ago
        self.ten = datetime(today.year-10,today.month,today.day) #ten years ago
        self.twenty = datetime(today.year-20,today.month,today.day) #twenty years ago
        
    def dividend_score(self, ticker):
        try:
            self.dp = 0
            t = yf.Ticker(ticker)   
            div = t.dividends
            div = div.to_frame()
            div = div.tz_localize(None)
            price = td.sdata(ticker, str(div.index[0])[0:10], self.today)

            if len(div) > 0:    

                divs = np.zeros(len(div))
                dive = np.zeros(len(div))
                dives = np.zeros(len(div))

                for i in range(1,len(div)):
                    divs[i] = div.iloc[i,0] / div.iloc[i-1,0]

                for i in range(1, len(div)):
                    dive[i] = div.iloc[i,0] / price.iloc[i]
                    if i > 1:
                        dives[i] = dive[i] / dive[i-1]

                if np.mean(divs) > 1.0:
                    self.dp = self.dp + 1

                if np.mean(dives) > 1.0:
                    self.dp = self.dp + 1
        except:
            self.dp = 0
            
        for i in range(len(self.scores)):
            if self.scores.index[i] == ticker:
                self.scores.iloc[i,2] = self.dp
        
    def holder_score(self,ticker):
        
        self.hp = 0
        t = yf.Ticker(ticker)   
        self.mh = t.major_holders
        self.ih = t.institutional_holders 
    
        if float(self.mh.iloc[0,0][0:3]) > 3:
            self.hp = self.hp + 1

        if float(self.mh.iloc[1,0][0:3]) > 50:
            self.hp = self.hp + 1

        if float(self.mh.iloc[2,0][0:3]) > 50:
            self.hp = self.hp + 1

        if float(self.mh.iloc[3,0]) > 1000:
            self.hp = self.hp + 1

        GIH = ["Vanguard Group Inc", "Blackrock Inc.", "Berkshire Hathaway, Inc", "State Street Corporation", "Morgan Stanley", "Norges Bank Investment Management", "JP Morgan Chase & Company", "Goldman Sachs Group Inc", "Bank of America Corporation", "Charles Schwab Investment Management, Inc.", "Bank Of New York Mellon Corporation", "Citadel Advisors Llc", "Dimensional Fund Advisors LP"]

        for i in range(len(GIH)):
            for j in range(len(self.ih['Holder'])):
                if GIH[i] == self.ih['Holder'][j]:
                    self.hp = self.hp + 1
        
        for i in range(len(self.scores)):
            if self.scores.index[i] == ticker:
                self.scores.iloc[i,1] = self.hp

    def financial_score(self,ticker):
        
        self.fp = 0
        t = yf.Ticker(ticker)    
        bs = t.balance_sheet
        fin = t.financials
        cs = t.cashflow
        for i in range(3):
            #
            if 'Net Debt' in bs.index:
                if bs[bs.columns[i]]['Net Debt'] / bs[bs.columns[i+1]]['Net Debt'] < 1:
                    self.fp = self.fp + 1
                elif bs[bs.columns[i]]['Net Debt'] / bs[bs.columns[i+1]]['Net Debt'] > 1:
                    self.fp = self.fp - 1
            #
            if bs[bs.columns[i]]['Total Debt'] / bs[bs.columns[i+1]]['Total Debt'] < 1:
                self.fp = self.fp + 1
            elif bs[bs.columns[i]]['Total Debt'] / bs[bs.columns[i+1]]['Total Debt'] > 1:
                self.fp = self.fp - 1
            #
            if 'Long Term Debt' in bs.index:
                if bs[bs.columns[i]]['Long Term Debt'] / bs[bs.columns[i+1]]['Long Term Debt'] < 1:
                    self.fp = self.fp + 1
                elif bs[bs.columns[i]]['Long Term Debt'] / bs[bs.columns[i+1]]['Long Term Debt'] > 1:
                    self.fp = self.fp - 1
            #
            if 'Current Liabilities' in bs.index:
                if bs[bs.columns[i]]['Current Liabilities'] / bs[bs.columns[i+1]]['Current Liabilities'] < 1:
                    self.fp = self.fp + 1
                elif bs[bs.columns[i]]['Current Liabilities'] / bs[bs.columns[i+1]]['Current Liabilities'] > 1:
                    self.fp = self.fp - 1
            #
            if 'Current Debt' in bs.index and bs[bs.columns[i+1]]['Current Debt'] > 0:
                if bs[bs.columns[i]]['Current Debt'] / bs[bs.columns[i+1]]['Current Debt'] < 1:
                    self.fp = self.fp + 1
                elif bs[bs.columns[i]]['Current Debt'] / bs[bs.columns[i+1]]['Current Debt'] > 1:
                    self.fp = self.fp - 1
            #
            if 'Total Assets' in bs.index:
                if bs[bs.columns[i]]['Total Assets'] / bs[bs.columns[i+1]]['Total Assets'] > 1:
                    self.fp = self.fp + 1
                elif bs[bs.columns[i]]['Total Assets'] / bs[bs.columns[i+1]]['Total Assets'] < 1:
                    self.fp = self.fp - 1
            #
            if 'Current Assets' in bs.index:
                if bs[bs.columns[i]]['Current Assets'] / bs[bs.columns[i+1]]['Current Assets'] > 1:
                    self.fp = self.fp + 1
                elif bs[bs.columns[i]]['Current Assets'] / bs[bs.columns[i+1]]['Current Assets'] < 1:
                    self.fp = self.fp - 1
            
            if 'Total Capitalization' in bs.index:
                if bs[bs.columns[i]]['Total Capitalization'] / bs[bs.columns[i+1]]['Total Capitalization'] > 1:
                    self.fp = self.fp + 1
                elif bs[bs.columns[i]]['Total Capitalization'] / bs[bs.columns[i+1]]['Total Capitalization'] < 1:
                    self.fp = self.fp - 1
                    
            if 'Total Liabilities Net Minority Interest' in bs.index:
                if bs[bs.columns[i]]['Total Liabilities Net Minority Interest'] / bs[bs.columns[i+1]]['Total Liabilities Net Minority Interest'] > 1:
                    self.fp = self.fp + 1
                elif bs[bs.columns[i]]['Total Liabilities Net Minority Interest'] / bs[bs.columns[i+1]]['Total Liabilities Net Minority Interest'] < 1:
                    self.fp = self.fp - 1

            if 'Net Income' in fin.index:
                if fin[fin.columns[i]]['Net Income'] / fin[fin.columns[i+1]]['Net Income'] > 1:
                    self.fp = self.fp + 1
                elif fin[fin.columns[i]]['Net Income'] / fin[fin.columns[i+1]]['Net Income'] < 1:
                    self.fp = self.fp - 1

            if 'Total Revenue' in fin.index:
                if fin[fin.columns[i]]['Total Revenue'] / fin[fin.columns[i+1]]['Total Revenue'] > 1:
                    self.fp = self.fp + 1
                elif fin[fin.columns[i]]['Total Revenue'] / fin[fin.columns[i+1]]['Total Revenue'] < 1:
                    self.fp = self.fp - 1

            if 'Operating Revenue' in fin.index:
                if fin[fin.columns[i]]['Operating Revenue'] / fin[fin.columns[i+1]]['Operating Revenue'] > 1:
                    self.fp = self.fp + 1
                elif fin[fin.columns[i]]['Operating Revenue'] / fin[fin.columns[i+1]]['Operating Revenue'] < 1:
                    self.fp = self.fp - 1
                    
            if 'Normalized Income' in fin.index:
                if fin[fin.columns[i]]['Normalized Income'] / fin[fin.columns[i+1]]['Normalized Income'] > 1:
                    self.fp = self.fp + 1
                elif fin[fin.columns[i]]['Normalized Income'] / fin[fin.columns[i+1]]['Normalized Income'] < 1:
                    self.fp = self.fp - 1

            if 'Issuance Of Debt' in cs.index:
                if str(cs[cs.columns[i]]['Issuance Of Debt']) == "NaN":
                    cs[cs.columns[i]]['Issuance Of Debt'] = 0

            if 'Long Term Debt Issuance' in cs.index:
                if str(cs[cs.columns[i]]['Long Term Debt Issuance']) == "NaN":
                    cs[cs.columns[i]]['Long Term Debt Issuance'] = 0

            if cs[cs.columns[i]]['Operating Cash Flow'] > cs[cs.columns[i+1]]['Operating Cash Flow']:
                self.fp = self.fp + 1

            if 'Long Term Debt Payments' in cs.index and 'Long Term Debt Issuance' in cs.index:
                if abs(cs[cs.columns[i]]['Long Term Debt Payments']) > 0.5 * cs[cs.columns[i]]['Long Term Debt Issuance']:
                    self.fp = self.fp + 1

            if 'Repayment Of Debt' in cs.index and 'Issuance Of Debt' in cs.index:
                if abs(cs[cs.columns[i]]['Repayment Of Debt']) > 0.5 * cs[cs.columns[i]]['Issuance Of Debt']:
                    self.fp = self.fp + 1
        
        for i in range(len(self.scores)):
            if self.scores.index[i] == ticker:
                self.scores.iloc[i,0] = self.fp
    
    def technical_score(self,ticker):
        
        self.tp = 0
        df = pd.DataFrame()
        t = yf.Ticker(ticker)
        try:
            df = t.history(start=self.twenty, end=self.today)['Close']
            df.index = pd.to_datetime(df.index).tz_localize(None)
        except:
            df = t.history(start=self.five, end=self.today)['Close']
            df.index = pd.to_datetime(df.index).tz_localize(None)


        self.means = []
        self.stds = []

        l = int(round(len(df.index)/5,0))

        for i in range(5):
            if i < 4:
                self.means.append(np.mean(df[df.index > df.index[l*(i+1)]]))
                self.stds.append(np.std(df[df.index > df.index[l*(i+1)]]))
            if i == 4:
                self.means.append(np.mean(df[df.index > df.index[l*(i+1) - 150]]))
                self.stds.append(np.std(df[df.index > df.index[l*(i+1) - 150]]))
        
        if self.stds[0] < self.stds[len(self.stds)-1] or self.stds[0] < self.stds[len(self.stds)-2]:
            self.tp = self.tp + 1
        if self.means[0] < self.means[len(self.means)-1] or self.means[0] < self.means[len(self.means)-2]:
            self.tp = self.tp + 1
            
        if (self.means[len(self.means)-1]-5) / self.stds[len(self.stds)-1] > 1:
            self.tp = self.tp + 1
        elif (self.means[len(self.means)-1]-5) / self.stds[len(self.stds)-1] < 1:
            self.tp = self.tp - 1
        
        for i in range(len(self.scores)):
            if self.scores.index[i] == ticker:
                self.scores.iloc[i,3] = self.tp
    
    def score(self):
        
        for ticker in self.portf:
            
            try:
                self.financial_score(ticker)
                self.holder_score(ticker)
                self.dividend_score(ticker)
                
            except:
                for i in range(len(self.scores)):
                    
                    if self.scores.index[i] == ticker:
                        
                        self.scores.iloc[i,0] = 0
                        self.scores.iloc[i,1] = 0
                        self.scores.iloc[i,2] = 0
                
            try:
                
                self.technical_score(ticker)
    
            except:
            
                for i in range(len(self.scores)):
                    
                    if self.scores.index[i] == ticker:
                        
                        self.scores.iloc[i,3] = 0
            
            for i in range(len(self.scores)):
                
                 if self.scores.index[i] == ticker:
                    
                    if self.scores.iloc[i,0] > 8.0:

                        self.scores.iloc[i,4] = self.scores.iloc[i,4] + 1

                    if self.scores.iloc[i,1] > 7.0:

                        self.scores.iloc[i,4] = self.scores.iloc[i,4] + 1

                    self.scores.iloc[i,4] = self.scores.iloc[i,4] + 0.6 * self.scores.iloc[i,2] + 0.4 * self.scores.iloc[i,3]

        def narrow_scores(self):
            self.scores = self.scores[self.scores['financials'] > np.mean(self.scores['financials'])]
            self.scores = self.scores[self.scores['holders'] > np.mean(self.scores['holders'])]
            self.scores = self.scores[self.scores['dividend'] > 0.0]         


    
class financial_analytics:
    
    def __init__(self):
        self.initials = "pp"
    
    def fractionated_interest(self, i,m):
        ir = m *((1+i)**(1/m)-1)
        return ir
    
    def summarized_interest(self, i,m):
        ir = (1+i/m)**m - 1
        return ir
    
    def continous_interest(self, i,t):
        cont = 2.7182818**(math.log(1+i)*t)
        return cont
    
    def continous_discounting(self, i,n,t):
        cont = 2.7182818**(-math.log(1+i)*(n-t))
        return cont
    
    def presentvalue_c(self, c,i,t):
        pv = c * (1+i)**t
        return pv
    
    def presentvalue_b(self, b,i,t):
        pv = b*(1/(1+i))**t
        return pv
    
    #time value of c
    def time_value_c(self, c,b,i):
        m = 0
        M = 0
        for j in range(0,len(c)):
            m = (c[j] - b[j])*(1+i[j])
            M = M+m
        return(M)
    
    #time value of v
    def time_value_v(self, c,b,i):
        m = 0
        M = 0
        for j in range(0,len(c)):
            m = (c[j] - b[j])*1/(1+i[j])
            M = M+m
        return(M)
    
    #essentially difference between timevalues
    def fair_stream(self, c,b,i,n):
        t = len(c)
        n = t
        U = 0
        V = 0
        for j in range(0,n):
            U = U + (c[j] - b[j])*(1+i[j])**(t-j)
            V = V + (b[j] - c[j])*(1+i[j])**(-(t+1-t))
        return(U+V)
    
    
    def yrlygrowth(self, total_growth, years):
        """
        Determine the annual growth from the growth over an
        arbitrary time span.
        """
        return math.exp(math.log(total_growth) / years)
    
    def pvannuity(self, option = "annual", rate = 1, nrmts = 1, amt=1):
        if option == "annual":
            amt * (1. - (1. + rate)**-nrmts) / rate
            
        elif option == "semi-annual":
            amt * (1. - (1. + rate)**-nrmts) / rate
            
        elif option == "quarter":
            amt * (1. - (1. + rate)**-nrmts) / rate
            
        return amt * (1. - (1. + rate)**-nrmts) / rate
    
    def loanpayment(self, amount, Interest_rate, nr_payments):
        return float(amount) / self.pvannuity(Interest_rate, nr_payments)
    
    
    def inst_to_ann(self, r):
        """
        Convert an instantaneous interest rate to an annual interest rate
        """
        return np.expm1(r)
    
    def ann_to_inst(self, r):
        """
        Convert an instantaneous interest rate to an annual interest rate
        """
        return np.log1p(r)
    
    # Define the interest rate simulation based on the CIR model
    def cir(self, n_years = 10, n_scenarios=1, a=0.05, b=0.03, sigma=0.05, steps_per_year=12, r_0=None):
        """
        Generate random interest rate evolution over time using the CIR model
        b and r_0 are assumed to be annualized rates
        output values are annualized rates as well
        """
        if r_0 is None: r_0 = b 
        r_0 = self.ann_to_inst(r_0)
        dt = 1 / steps_per_year
        num_steps = int(n_years * steps_per_year) + 1 # because n_years might be a float
        
        shock = np.random.normal(0, scale=np.sqrt(dt), size=(num_steps, n_scenarios))
        rates = np.empty_like(shock)
        rates[0] = r_0
        for step in range(1, num_steps):
            r_t = rates[step-1]
            d_r_t = a*(b-r_t)*dt + sigma*np.sqrt(r_t)*shock[step]
            rates[step] = abs(r_t + d_r_t) # just in case of roundoff errors going negative
            
        return pd.DataFrame(data=self.inst_to_ann(rates), index=range(num_steps))
    
    
    def annuity_in_arrear(self, v0, t, i):
        res = (1-(1/(1+i))**t)/i
        return res
    
    def annuity_due(self, v0,t,i):
        res = (1-(1/(1+i))**t)/(1-(1/(1+i)))
        return res
    
    # needs correcting
    def deferred_annuity_in_arrear(self, v0, t, k, i):
        if t < k:
            (1/(1+i))**k * self.annuity_in_arrear(v0,k,i)
        elif t >= k:
            self.annuity_in_arrear(v0,t,i) - self.annuity_in_arrear(v0, k ,i)
            
    def deferred_annuity_due(self, v0,t,k,i):
        if t < k:
            (1/(1+i))**k * self.annuity_in_due(v0,k,i)
        elif t >= k:
            self.annuity_in_arrear(v0,t,i) - self.annuity_in_due(v0, k ,i)
            
    def cont_a(self, i,n):
        a = (1 - self.cont_dis(i,n,0))/math.log(1+i)
        return a
    
    def cont_def_a(self, i,n,k):
        a_n = (1 - self.cont_dis(i,n,0))/math.log(1+i)
        a_k = (1 - self.cont_dis(i,k,0))/math.log(1+i)
        return a_n - a_k



            

import math, random;
def median(data, l=0):
    l = (l) if l > 0 else len(data); pl = []; med = [];
    for i in range(len(data)):
        pl.append(data[i]);
        if (len(pl)) >= l:
            tmp = pl[:];
            tmp.sort(reverse=True);
            med.append(tmp[int(round((len(tmp)) / 2))]);
            pl = pl[1:];
    return med;
def kmeans(data, clusters):
    means = []; centers = []; old = []; changed = True; init = round(len(data)/(clusters+1));
    for i in range(clusters): centers.append([]);
    for i in range(clusters): centers[i] = data[init*(i+1)];
    while(changed):
        for i in range(clusters): means.append([]);
        changed = False;
        for x in range(len(data)):
            rang = -1; oldrange = -1;
            for y in range(clusters):
                r = abs(centers[y]-data[x]);
                if(oldrange == -1):
                    oldrange = r;
                    n = y;
                elif(r <= oldrange):
                    oldrange = r;
                    n = y;
            means[n].append(data[x]);
        old = centers;
        for x in range(clusters):
            sum = 0;
            for y in range(len(means[x])): sum += means[x][y];
            m = sum / len(means);
            centers[x] = m;
        for x in range(clusters):
            if centers[x] != old[x]: changed = True;
    return means;
def normalize(data, marg=0):
    ma = max(data)*(1+marg); mi = min(data)*(1-marg); norm = [];
    for i in range(0, len(data)):
        norm.append(1-(ma-data[i])/(ma-mi));
    return norm;
def denormalize(data, norm, marg=0):
    ma = max(data)*(1+marg); mi = min(data)*(1-marg); dnorm = [];
    for i in range(0, len(norm)):
        dnorm.append(mi+norm[i]*(ma-mi));
    return dnorm;
def standardize(data):
    mean = sma(data, len(data));
    st = std(data); res = [];
    for i in range(len(data)):
        res.append((data[i]-mean[0])/st);
    return res;
def mad(data, l=0):
    l = l if l > 0 else len(data); med = [];
    for i in range(l, len(data)+1):
        tmp = data[i-l:i];
        m = median(data[i-l:i]); adev = [];
        for q in range(len(tmp)): adev.append(abs(float(tmp[q]) - float(m[len(m)-1])));
        ad = median(adev);
        med.append(ad[len(ad)-1]);
    return med;
def aad(data, l=0):
    l = l if l > 0 else len(data); med = [];
    for i in range(l, len(data)+1):
        tmp = data[i-l:i];
        m = sma(tmp, l); sum = 0;
        for q in range(len(tmp)): sum += abs(tmp[q] - m[len(m)-1]);
        med.append(sum/l);
    return med;
def ssd(data, l=0):
    l = l if l > 0 else len(data); sd = [];
    for i in range(l, len(data)+1):
        tmp = data[i-l:i]; mean = sma(tmp, l); sum = 0;
        for q in range(len(tmp)): sum += (tmp[q] - mean[0])**2;
        sd.append(sum**(1/2));
    return sd;
def er(data):
    wins = []; losses = []; wp = 1; lp = 1;
    for i in range(len(data)):
        if data[i] >= 0:
            wins.append(data[i]+1);
        else:
            losses.append(data[i]+1);
    win = (len(wins) / len(data)); loss = (len(losses) / len(data));
    for i in range(len(losses)):
        lp *= losses[i];
    for i in range(len(wins)):
        wp *= wins[i];
    return (((wp**(1/len(wins))-1)*100) * win + ((lp**(1/len(losses))-1)*100)*loss) / 100
def rsi(data, l=14):
    pl = []; rs = [];
    for i in range(1, len(data)):
        pl.append(data[i] - data[i - 1]);
        if (len(pl)) >= l-1:
            gain = 0.0; loss = 0.0;
            for a in range(len(pl)):
                if pl[a] > 0: gain += float(pl[a]);
                if pl[a] < 0: loss += abs(float(pl[a]));
            gain /= l; loss /= l;
            try:
                f = 100.0 - 100.0 / (1.0 + (gain/loss));
            except:
                f = 100.0
            rs.append(f);
            pl = pl[1:];
    return rs;
def wrsi(data, l=14):
    arrsi = []; u = []; d = [];
    for i in range(1,len(data)):
        if(data[i]-data[i-1]<0):
            d.append(abs(data[i]-data[i-1]));
            u.append(0.0);
        else:
            d.append(0.0);
            u.append(data[i]-data[i-1]);
    d = wsma(d, l); u = wsma(u, l);
    for i in range(0, len(d)):
        try:
            f = 100.0-100.0/(1.0+(u[i]/d[i]));
        except:
            f = 100.0;
        arrsi.append(f);
    return arrsi;
def sma(data, l=14):
    pl = []; sm = [];
    for i in range(len(data)):
        pl.append(float(data[i]));
        if (len(pl)) >= l:
            sm.append(sum(pl) / l);
            pl = pl[1:];
    return sm;
def smma(data, l=14):
    pl = []; sm = [];
    for i in range(len(data)):
        pl.append(float(data[i]));
        if (len(pl)) >= l:
            average = 0;
            for a in range(len(pl)): average += pl[a];
            f = average / l if len(sm) <= 0 else (average - sm[len(sm) - 1]) / l;
            sm.append(f);
            pl = pl[1:];
    sm = sm[1:];
    return sm;
def wma(data, l=14):
    pl = []; wm = []; weight = [];
    for i in range(1, l+1):
        weight.append(i);
    for i in range(len(data)):
        pl.append(float(data[i]));
        if (len(pl)) >= l:
            average = 0;
            for q in range(1, len(pl)+1):
                average += pl[q-1] * q / sum(weight);
            wm.append(average);
            pl = pl[1:];
    return wm;
def wsma(data, l=14):
    em = []; weight = 1/l;
    for i in range(l, len(data)+1):
        if len(em) > 0:
            em.append((data[i-1]-em[len(em)-1]) * weight + em[len(em)-1]);
            continue;
        pl = data[i-l:i]; average = 0;
        for q in range(0, len(pl)):
            average += pl[q];
        em.append(average/len(pl));
    return em;
def pwma(data, l=14):
    weight = 0; wmaa = []; weights = []; b = l;
    for i in range(-round(l/2), 0):
        if(abs(i) % 1 != 0):
            i = round(abs(i));
            weight += (i*b);
        else:
            weights.append(abs(i)*b);
            weight += (abs(i)*b*2);
        weights.insert(0,abs(i)*b);
        b -= 1;
    for i in range(l, len(data)+1):
        average = 0; pl = data[i-l:i];
        for x in range(0, len(weights)):
            average += pl[x] * weights[x] / weight;
        wmaa.append(average);
    return wmaa;
def hwma(data, l=14):
    weight = 0; wmaa = []; weights = []; b = l;
    for i in range(1, round(l/2+1)):
        if(i%1 != 0):
            i = round(i);
            weight += (i*b);
        else:
            weights.append(i*b);
            weight += (i*b*2)
        weights.insert(0,i*b);
        b -= 1;
    for i in range(l, len(data)+1):
        average = 0; pl = data[i-l:i];
        for x in range(len(weights)):
            average += pl[x] * weights[x] / weight;
        wmaa.append(average);
    return wmaa;
def ema(data, l=12):
    pl = []; em = []; weight = 2 / (float(l) + 1);
    for i in range(len(data)):
        pl.append(float(data[i]));
        if (len(pl)) >= l:
            f = sum(pl) / l if len(em) <= 0 else (data[i] - em[len(em) - 1]) * weight + em[len(em) - 1]
            em.append(f);
            pl = pl[1:];
    return em;
def tsi(data, long=25, short=13, sig=13):
    mo = []; ab = []; ts = []; tsi = [];
    for i in range(1, len(data)):
        mo.append(data[i] - data[i - 1]);
        ab.append(abs(data[i] - data[i - 1]));
    sma1 = ema(mo, long); sma2 = ema(ab, long);
    ema1 = ema(sma1, short); ema2 = ema(sma2, short);
    for i in range(len(ema1)):
        ts.append(ema1[i] / ema2[i]);
    tma = ema(ts, sig);
    ts = ts[(len(ts)-len(tma)):];
    for i in range(len(tma)):
        tsi.append([tma[i], ts[i]]);
    return tsi;
def vwma(data, l=20):
    pl = []; wm = [];
    for i in range(len(data)):
        pl.append(data[i]);
        if (len(pl)) >= l:
            weight = 0; sum = 0;
            for a in range(len(pl)):
                sum += (pl[a][0] * pl[a][1]);
                weight += pl[a][1];
            wm.append(sum/weight);
            pl = pl[1:];
    return wm;
def hull(data, l=14):
    pl = []; hma = []; ewma = wma(data[:], l); sqn = round(l**(1.0/2.0));
    first = wma(data[:], int(round(l / 2)));
    first = first[-(len(ewma)-len(first)):];
    for i in range(len(ewma)):
        pl.append((first[i] * 2) - ewma[i]);
        if (len(pl)) >= sqn:
            h = wma(pl[:], int(sqn));
            hma.append(h[len(h) - 1]);
            pl = pl[1:];
    return hma;
def kama(data, l1=10, l2=2, l3=30):
    ka = sma(data[:], l1); ka = [ka[len(ka)-1]];
    for i in range(l1 + 1, len(data)):
        vola = 0; change = abs(data[i] - data[i - l1]);
        for a in range(1, l1):
            vola += abs(float(data[i-a]) - float(data[(i-a)-1]));
        sc = (change/vola * (2.0/(l2+1.0) - 2.0/(l3+1.0) + 2.0/(l3+1.0))) ** 2.0;
        ka.append(ka[len(ka)-1] + sc * (float(data[i]) - ka[len(ka)-1]));
    return ka;
def macd(data, l1=12, l2=26):
    if l1 > l2: [l1, l2] = [l2, l1];
    em1 = ema(data[:], l1); em2 = ema(data[:], l2);
    em1 = em1[-(len(em2) - len(em1)):]; emf = [];
    for i in range(len(em1)):
        emf.append(em1[i] - em2[i]);
    return emf;
def variance(data, l1=0):
    l1 = l1 if l1 > 0 else len(data); va = [];
    for i in range(l1, len(data)+1):
        tmp = data[i-l1:i]; mean = sma(tmp, len(tmp)); sum = 0;
        for x in range(len(tmp)): sum += ((tmp[x] - mean[len(mean)-1]) ** 2);
        va.append(sum/l1);
    return va;
def std(data, l1=0):
    l1 = (l1) if l1 > 0 else len(data); v = variance(data[:], l1);
    std = v[len(v)-1] ** (1.0/2.0)
    return std;
def normsinv(p):
    a1 = -39.6968302866538; a2 = 220.946098424521; a3 = -275.928510446969; a4 = 138.357751867269; a5 = -30.6647980661472; a6 = 2.50662827745924;
    b1 = -54.4760987982241; b2 = 161.585836858041; b3 = -155.698979859887; b4 = 66.8013118877197; b5 = -13.2806815528857; c1 = -7.78489400243029E-03;
    c2 = -0.322396458041136; c3 = -2.40075827716184; c4 = -2.54973253934373; c5 = 4.37466414146497; c6 = 2.93816398269878; d1 = 7.78469570904146E-03;
    d2 = 0.32246712907004; d3 = 2.445134137143; d4 = 3.75440866190742; p_low = 0.02425; p_high = 1 - p_low;
    if p < 0 or p > 1:
        return 0;
    if p < p_low:
        q = math.sqrt(-2*math.log(1-p));
        return (((((c1 * q + c2) * q + c3) * q + c4) * q + c5) * q + c6) / ((((d1 * q + d2) * q + d3) * q + d4) * q + 1);
    if p <= p_high:
        q = p - 0.5;
        r = q * q;
        return (((((a1 * r + a2) * r + a3) * r + a4) * r + a5) * r + a6) * q / (((((b1 * r + b2) * r + b3) * r + b4) * r + b5) * r + 1);
    q = math.sqrt(-2*math.log(1-p));
    return -(((((c1 * q + c2) * q + c3) * q + c4) * q + c5) * q + c6) / ((((d1 * q + d2) * q + d3) * q + d4) * q + 1);
def sim(data, l=50, sims=1000, perc=-1):
    sd = [];
    for i in range(sims):
        projected = data[:];
        for x in range(l):
            change = [];
            for y in range(1, len(projected)):
                change.append((projected[y]-projected[y-1])/projected[y-1]);
            mean = sma(change, len(change));
            st = std(change); rando = normsinv(random.random());
            projected.append(projected[len(projected)-1]*math.exp(mean[0]-(st*st)/2+st*rando));
        sd.append(projected);
    if perc <= -1: return sd;
    finalprojection = data[:];
    for i in range(len(sd[0])):
        so = [];
        for x in range(len(sd)):
            so.append(sd[x][i])
        so.sort();
        finalprojection.append(so[round((len(so)-1)*perc)]);
    return finalprojection;
def percentile(data, perc=0.5):
    final = [];
    for i in range(len(data[0])):
        tmp = [];
        for x in range(len(data)):
            tmp.append(data[x][i]);
        tmp.sort();
        final.append(tmp[round((len(tmp)-1)*perc)])
    return final;
def bands(data, l1=14, l2=1):
    pl = []; deviation = []; boll = [];
    sm = sma(data[:], l1);
    for i in range(len(data)):
        pl.append(data[i]);
        if (len(pl)) >= l1:
            devi = std(pl[:], l1);
            deviation.append(devi);
            pl = pl[1:];
    for i in range(len(sm)):
        boll.append([sm[i] + deviation[i] * l2, sm[i], sm[i] - deviation[i] * l2]);
    return boll;
def bandwidth(data, l1=14, l2=1):
    band = bands(data[:], l1, l2); boll = [];
    for i in range(len(band)): boll.append((band[i][0] - band[i][2]) / band[i][1]);
    return boll;
def atr(data, l1=14):
    atr = [float(data[0][0]) - float(data[0][2])];
    for i in range(1, len(data)):
        t0 = max((float(data[i][0]) - float(data[i - 1][1])), (float(data[i][2]) - float(data[i - 1][1])), (float(data[i][0]) - float(data[i][2])));
        atr.append((atr[len(atr)-1] * (l1 - 1) + t0) / l1)
    return atr;
def keltner(data, l1=14, l2=1):
    closing = []; tr = atr(data[:], l1); kelt = [];
    for i in range(len(data)): closing.append(sum(data[i]) / 3.0);
    kma = sma(closing, l1);
    tr = tr[(len(tr) - len(kma)):];
    for i in range(len(kma)): kelt.append([kma[i] + tr[i] * l2, kma[i], kma[i] - tr[i] * l2]);
    return kelt
def cor(data1, data2):
    d1avg = sma(data1[:], len(data1)); d1avg = d1avg[len(d1avg)-1];
    d2avg = sma(data2[:], len(data2)); d2avg = d2avg[len(d2avg)-1]
    sumavg = 0; sx = 0; sy = 0;
    for i in range(len(data1)):
        x = data1[i]-d1avg;
        y = data2[i]-d2avg;
        sumavg += (x * y); sx += x**2; sy += y**2;
    n = len(data1)-1;
    sx/=n; sy/=n; sx = sx ** (1/2); sy = sy ** (1/2);
    return (sumavg / (n*sx*sy));
def dif(n, o): return (n-o)/o;
def drawdown(d):
    max = d[0]; min = d[0]; big = 0;
    for y in range(1,len(d)):
        if(d[y] > max):
            if(min != 0):
                diff = dif(min, max);
                if(diff < big): big = diff;
                min = d[y];
            max = d[y];
        if(d[y] < min): min = d[y];
    diff = dif(min, max);
    if(diff < big): big = diff;
    return big;
def aroon_up(data, l1=10):
    pl = []; aroon = [];
    for i in range(len(data)):
        pl.append(float(data[i]));
        if(len(pl) >= l1):
            hl = pl[:];
            aroon.append((100.0 * (l1-1-pl.index(max(hl))) / (l1-1)) if l1-1 != 0 else 0);
            pl = pl[1:];
    return aroon;
def aroon_down(data, l1=10):
    pl = []; aroon = [];
    for i in range(len(data)):
        pl.append(float(data[i]));
        if(len(pl) >= l1):
            hl = pl[:];
            hl.reverse();
            aroon.append(100.0 * (l1-1-hl.index(min(hl))) / (l1-1) if l1-1 != 0 else 0);
            pl = pl[1:];
    return aroon;
def aroon_osc(data, l1=10):
    pl = []; aroon = [];
    u = aroon_up(data[:], l1); d = aroon_down(data[:], l1);
    for i in range(len(u)): aroon.append(u[i] - d[i]);
    return aroon;
def mfi(data, l1=14):
    mfi = []; n = []; p = [];
    for i in range(len(data)):
        p.append(float(data[i][0])); n.append(float(data[i][1]));
        if(len(p) >= l1):
            mfi.append((100.0 - 100.0 / (1 + sum(p) / sum(n))));
            p = p[1:]; n = n[1:];
    return mfi;
def roc(data, l1=14):
    pl = []; roc = [];
    for i in range(len(data)):
        pl.append(float(data[i]));
        if(len(pl) >= l1):
            roc.append((pl[len(pl)-1] - pl[0]) / pl[0]);
            pl = pl[1:];
    return roc;
def cop(data, l1=11, l2=14, l3=10):
    m = max(l1, l2); co = [];
    if l1 > l2: [l1, l2] = [l2, l1];
    for i in range(m + l3, len(data)):
        r1 = roc(data[i-(m+l3):i], l1); r2 = roc(data[i-(m+l3):i], l2); tmp = [];
        r1 = r1[(len(r1) - len(r2)):];
        for a in range(len(r1)):
            tmp.append(r1[a] + r2[a]);
        tmp = wma(tmp[:], l3);
        co.append(tmp[len(tmp)-1]);
    return co;
def kst(data, r1=10, s1=10, r2=15, s2=10, r3=20, s3=10, r4=30, s4=15, sig=9):
    ks = []; fs = []; ms = (max(r1, r2, r3, r4) + max(s1, s2, s3, s4));
    for i in range(ms, len(data)+1):
        rcma1 = roc(data[i-ms:i], r1); rcma2 = roc(data[i-ms:i], r2); rcma3 = roc(data[i-ms:i], r3); rcma4 = roc(data[i-ms:i], r4);
        rcma1 = sma(rcma1, s1); rcma2 = sma(rcma2, s2); rcma3 = sma(rcma3, s3); rcma4 = sma(rcma4, s4);
        ks.append(rcma1[len(rcma1)-1] + rcma2[len(rcma2)-1] + rcma3[len(rcma3)-1] + rcma4[len(rcma4)-1]);
    sl = sma(ks[:], sig);
    ks = ks[(len(ks) - len(sl)):];
    for i in range(len(ks)): fs.append([ks[i], sl[i]]);
    return fs;
def obv(data):
    obv = [0];
    for i in range(1, len(data)):
        if(data[i][1] > data[i-1][1]): obv.append(obv[len(obv)-1] + data[i][0]);
        if(data[i][1] < data[i-1][1]): obv.append(obv[len(obv)-1] - data[i][0]);
        if(data[i][1] == data[i-1][1]): obv.append(obv[len(obv)-1]);
    return obv;
def vwap(data, l1=0):
    l1 = l1 if l1 > 0 else len(data); pl = []; vwap = [];
    for i in range(len(data)):
        pl.append([(data[i][0] * data[i][1]), data[i][1]]);
        if(len(pl) >= l1):
            totalv = 0; totalp = 0;
            for a in range(len(pl)):
                totalv += pl[a][1];
                totalp += pl[a][0];
            vwap.append(totalp/totalv);
            pl = pl[1:];
    return vwap;
def mom(data, l1=10, p=False):
    mom = [];
    for i in range(l1-1, len(data)):
        mom.append(data[i] / data[i - (l1-1)] * 100) if p == True else mom.append(data[i] - data[i - (l1 - 1)]);
    return mom
def mom_osc(data, l1=9):
    pl = []; osc = []; l1+=1;
    for i in range(len(data)):
        pl.append(data[i]);
        if(len(pl) >= l1):
            sumh = 0; suml = 0;
            for a in range(1, l1):
                if(pl[a-1] < pl[a]): sumh = sumh+pl[a];
                else: suml = suml+pl[a];
            osc.append((sumh - suml) / (sumh + suml) * 100);
            pl = pl[1:];
    return osc;
def bop(data, l1=14):
    bo = [];
    for i in range(len(data)):
        bo.append((data[i][3] - data[i][0]) / (data[i][1] - data[i][2]) if (data[i][1] - data[i][2]) != 0 else 0);
    bo = sma(bo, l1);
    return bo;
def fi(data, l1=13):
    pl = []; ff = [];
    for i in range(1, len(data)):
        pl.append((data[i][0] - data[i-1][0]) * data[i][1]);
        if(len(pl) >= l1):
            vfi = ema(pl[:], l1);
            ff.append(vfi[len(vfi)-1]);
            pl = pl[1:];
    return ff;
def asi(data):
    pl = []; a = [];
    for i in range(1, len(data)):
        c = float(data[i][1]); y = float(data[i-1][1]); h = float(data[i][0]); hy = float(data[i-1][0]); cy = float(data[i-1][1]);
        l = float(data[i][2]); ly = float(data[i-1][2]); o = float(data[i][0]); oy = float(data[i-1][0]); t = max(float(data[i][0]), float(data[i-1][0])) - min(float(data[i][2]), float(data[i-1][2]));
        if(hy-c > ly-c): k = hy-c;
        else: k = ly-c;
        if((h - cy > l - cy) & (h - cy > h - l)): r = h - cy - (l - cy) / 2.0 + (cy - oy) / 4.0;
        if((l - cy > h - cy) & (l - cy > h - l)): r = l - cy - (h - cy) / 2.0 + (cy - oy) / 4.0;
        if((h - l > h - cy) & (h - l > l - cy)): r = h - l + (cy - oy) / 4.0;
        a.append(50.0 * ((cy - c + (cy - oy) / 2.0 + (c - o) / 2.0) / r) * k / t if r and t != 0 else 0);
    return a;
def ao(data, l1=5, l2=35):
    pl = []; a = [];
    for i in range(len(data)):
        pl.append((float(data[i][0]) + float(data[i][1])) / 2.0);
        if(len(pl) >= l2):
            f = sma(pl[:], l1);
            s = sma(pl[:], l2);
            a.append(f[len(f)-1] - s[len(s)-1]);
            pl = pl[1:];
    return a;
def pr(data, l1=14):
    n = []; pl = [];
    for i in range(len(data)):
        pl.append(float(data[i]));
        if(len(pl) >= l1):
            highd = max(pl[:]); lowd = min(pl[:]);
            n.append((highd - data[i]) / (highd - lowd) * -100.0 if highd-lowd != 0 else 0);
            pl = pl[1:];
    return n;
def lsma(data, l1=25):
    pl = []; lr = [];
    for i in range(len(data)):
        pl.append(float(data[i]));
        if(len(pl) >= l1):
            sum_x = 0.0; sum_y = 0.0; sum_xy = 0.0; sum_xx = 0.0; sum_yy = 0.0;
            for a in range(1, len(pl)+1):
                sum_x += a;
                sum_y += pl[a-1];
                sum_xy += (pl[a-1] * a);
                sum_xx += (a*a);
                sum_yy += (pl[a-1] * pl[a-1]);
            m = ((sum_xy - sum_x * sum_y / l1) / (sum_xx - sum_x * sum_x / l1));
            b = sum_y / l1 - m * sum_x / l1;
            lr.append(m * l1 + b);
            pl = pl[1:];
    return lr;
def don(data, l1=20):
    pl = []; channel = [];
    for i in range(len(data)):
        pl.append(data[i]);
        if(len(pl) >= l1):
            highs = []; lows = [];
            for h in range(len(pl)):
                highs.append(float(pl[h][0]));
                lows.append(float(pl[h][1]));
            m = max(highs); l = min(lows);
            channel.append([m, (m+l) / 2.0, l]);
            pl = pl[1:];
    return channel;
def ichimoku(data, l1=9, l2=26, l3=52, l4=26):
    cloud = []; place = []; pl = [];
    for i in range(len(data)):
        pl.append(data[i]);
        if(len(pl) >= l3):
            highs = []; lows = [];
            for a in range(len(pl)):
                highs.append(float(pl[a][0]));
                lows.append(float(pl[a][2]));
            tsen = (max(highs[len(highs)-l1:len(highs)]) + (min(lows[len(lows)-l1:len(lows)]))) / 2.0;
            ksen = (max(highs[len(highs)-l2:len(highs)]) + (min(lows[len(lows)-l2:len(lows)]))) / 2.0;
            senka = float(data[i][1]) + ksen;
            senkb = (max(highs[len(highs)-l3:len(highs)]) + (min(lows[len(lows)-l2:len(lows)]))) / 2.0;
            chik = float(data[i][1]);
            place.append([tsen, ksen, senka, senkb, chik]);
            pl = pl[1:];
    for i in range(l4, len(place)-l4):
        cloud.append([place[i][0], place[i][1], place[i+l4][2], place[i+l4][3], place[i-l4][4]]);
    return cloud;
def stoch(data, l1=14, sd=3, sk=3):
    stoch = []; high = []; low = []; ka = [];
    for i in range(len(data)):
        high.append(float(data[i][0]));
        low.append(float(data[i][2]));
        if(len(high) >= l1):
            highd = max(high); lowd = min(low);
            k = 100.0 * (data[i][1] - lowd) / (highd - lowd) if highd - lowd != 0 else 0;
            ka.append(k);
        if(sk > 0 & len(ka) > sk):
            smoothedk = sma(ka[:], sk);
            ka.push(smoothedk[len(smoothedk)-1]);
        if(len(ka) - sk >= sd):
            d = sma(ka[:], sd);
            stoch.append([k, d[len(d)-1]]);
            high = high[1:];
            low = low[1:];
            ka = ka[1:];
    return stoch;
def ha(data):
    h = [[(float(data[0][0]) + float(data[0][3])) / 2, float(data[0][1]), float(data[0][2]), (float(data[0][0]) + float(data[0][1]) + float(data[0][2]) + float(data[0][3])) / 4.0]];
    for i in range(len(data)):
        h.append([(h[len(h)-1][0] + h[len(h)-1][3]) / 2, max(h[len(h)-1][0], h[len(h)-1][3], float(data[i][1])), min(h[len(h)-1][0], h[len(h)-1][3], float(data[i][2])), (float(data[i][0]) + float(data[i][1]) + float(data[i][2]) + float(data[i][3])) / 4.0]);
    return h;
def ren(data, bs=1):
    re = []; decimals = len(str(bs-int(bs))[1:]);
    bh = round(round(float(data[0][0]) / float(bs) * (10.0 ** decimals)) / (10.0 ** decimals) * float(bs)); bl = bh - bs;
    for i in range(1, len(data)):
        if(data[i][0] > bh + bs):
            while (data[i][0] > bh + bs):
                re.append([bh,bh+bs,bh,bh+bs]);
                bh+=bs;
                bl+=bs;
        elif (data[i][1] < bl - bs):
            while (data[i][1] < bl - bs):
                re.append([bl,bl,bl-bs,bl-bs]);
                bh-=bs;
                bl-=bs;
    return re;
def envelope(data, l1=10, p=0.005):
    enve = [];
    for i in range(l1, len(data)):
        sm = sma(data[i-l1:i], l1);
        enve.append([sm[0]*(1+p),sm[0],sm[0]*(1-p)]);
    return enve;
def chaikin_osc(data, ema1=3, ema2=10):
    cha = []; adl = [];
    for i in range(len(data)):
        mfm = ((data[i][1]-data[i][2])-(data[i][0]-data[i][1]))/(data[i][0]-data[i][2]) if data[i][0]-data[i][2] != 0 else 0;
        adl.append(mfm*data[i][3]);
    ef = ema(adl, ema1); es = ema(adl, ema2);
    if len(ef) > len(es):
        ef = ef[len(ef)-len(es):];
    else:
        es = es[len(es)-len(ef):];
    for i in range(len(ef)):
        cha.append(ef[i]-es[i]);
    return cha;
def fractals(data):
    fractals = [[False, False], [False, False]];
    for i in range(2, len(data)-2):
        up = True if (data[i-2][0] < data[i][0] and data[i-1][0] < data[i][0] and data[i][0] > data[i+1][0] and data[i][0] > data[i+2][0]) else False
        down = True if (data[i-2][1] > data[i][1] and data[i-1][1] > data[i][1] and data[i][1] < data[i+1][1] and data[i][1] < data[i+2][1]) else False
        fractals.append([up, down]);
    fractals.append([False, False]);
    fractals.append([False, False]);
    return fractals;
def alligator(data, jl=13, tl=8, ll=5, js=8, ts=5, ls=3):
    ret = []; jaw = smma(data, jl); teeth = smma(data, tl); lips = smma(data, ll);
    teeth = teeth[len(teeth)-len(jaw):];
    lips = lips[len(lips)-len(jaw):];
    for i in range(len(jaw)-1, (js-2), -1):
        ret.append([jaw[i-(js-1)], teeth[i-(ts-1)], lips[i-(ls-1)]]);
    return ret;
def gator(data, jl=13, tl=8, ll=5, js=8, ts=5, ls=3):
    ret = []; jaw = smma(data, jl); teeth = smma(data, tl); lips = smma(data, ll);
    teeth = teeth[len(teeth)-len(jaw):];
    lips = lips[len(lips)-len(jaw):];
    for i in range(len(jaw)-1, (js-2), -1):
        ret.append([jaw[i-(js-1)]-teeth[i-(ts-1)], -(abs(teeth[i-(ts-1)]-lips[i-(ls-1)]))]);
    return ret;
def recent_high(data, lb=25):
    xback = lb; hindex = 0; highest = data[len(data)-1];
    for i in range(len(data)-2, 0, -1):
        if data[i] >= highest and xback > 0:
            highest = data[i];
            hindex = i;
            xback = lb;
        else:
            xback -= 1;
        if xback <= 0: break;
    return {"index": hindex, "value": highest};
def recent_low(data, lb=25):
    xback = lb; lindex = 0; lowest = data[len(data)-1];
    for i in range(len(data)-2, 0, -1):
        if data[i] <= lowest and xback > 0:
            lowest = data[i];
            lindex = i;
            xback = lb;
        else:
            xback -= 1;
        if xback <= 0: break;
    return {"index": lindex, "value": lowest};
def support(d, hl=0):
    hl = hl if hl else recent_low(d);
    findex = False; lowform = hl['value'];
    while findex == False:
        for i in range(hl['index'], len(d)):
            if hl['index']-i == 0: continue;
            newlow = (hl['value']-d[i])/(hl['index']-i);
            if newlow < lowform:
                lowform = newlow;
                index2 = i;
        if hl['index'] + 1 == index2 and index2 != len(d):
            hl['index'] = index2;
            lowform = min(d[:]);
            hl['value'] = d[hl['index']];
            findex = False;
        else:
            findex = True;
        if hl['index'] == len(d)-1: findex = True;
    if index2 == len(d)-1 or hl['index'] == len(d)-1:
        def calculate(pos):
            return pos*0+hl['value'];
        return {"calculate": calculate, "slope": 0, "lowest": hl['value'], "index": hl['index']};
    else:
        def calculate(pos):
            return pos*lowform+hl['value']
        return {"calculate": calculate, "slope": lowform, "lowest": hl['value'], "index": hl['index']};
def resistance(d, hl=0):
    hl = hl if hl else recent_high(d);
    findex = False; highform = hl['value'];
    while findex == False:
        for i in range(hl['index'], len(d)):
            if hl['index']-i == 0: continue;
            newhigh = (d[i]-hl['value'])/(hl['index']-i);
            if newhigh < highform:
                highform = newhigh
                index2 = i;
        if hl['index']+1 == index2 and index2 != len(d)-1:
            hl['index'] = index2;
            highform = max(d[:]);
            hl['value'] = d[hl['index']];
            findex = False;
        else:
            findex = True;
        if hl['index'] == len(d)-1: findex = True;
    if index2 == len(d)-1 or hl['index'] == len(d)-1:
        highform = 0;
    if index2 == len(d)-1 or hl['index'] == len(d)-1:
        def calculate(pos):
            return pos*0+hl['value'];
        return {"calculate": calculate, "slope": 0, "highest": hl['value'], "index": hl['index']}
    else:
        def calculate(pos):
            return pos*-highform+hl['value'];
        return {"calculate": calculate, "slope": highform, "highest": hl['value'], "index": hl['index']};
def ac(data, l1=5, l2=35):
    pl = [];
    for i in range(len(data)):
        pl.append((data[i][0]+data[i][1])/2);
    a = ao(data, l1, l2); sm =sma(a, l1); acr = [];
    if len(a) > len(sm):
        a = a[len(a)-len(sm):]
    else:
        sm = sm[len(sm)-len(a):];
    for i in range(len(a)): acr.append(a[i]-sm[i]);
    return acr;
def fib(start, end):
    return [start, (end-start)*.236+start, (end-start)*.382+start, (end-start)*.5+start, (end-start)*.618+start, (end-start)*.786+start, end, (end-start)*1.618+start, (end-start)*2.618+start, (end-start)*3.618+start, (end-start)*4.236+start]
def fisher(data, l=20):
    out = []; fish = 0; v1 = 0;
    for i in range(l, len(data)):
        pl = data[i-l:i]; pf = fish;
        mn = min(pl[:]);
        v1 = .33*2*((data[i]-mn)/(max(pl[:])-mn)-.5)+.67*v1;
        if v1 > 0.99: v1 = 0.999
        if v1 < -0.99: v1 = -0.999
        fish = 0.5 * math.log((1+v1)/(1-v1)) + 0.5 * pf;
        out.append([fish,pf]);
    return out[1:];
def ar(data, l):
    out = [];
    for i in range(l, len(data)):
        exp = er(data[i-l:i]);
        out.append(data[i]-exp);
    return out;
def winratio(data):
    wins = 0; losses = 0;
    for i in range(len(data)):
        if data[i] >= 0:
            wins += 1;
        else:
            losses += 1;
    return wins / (losses + wins)

def avgwin(data):
    wins = [];
    for i in range(len(data)):
        if data[i] >= 0: wins.append(data[i]);
    return wins / (losses + wins);

def avgwin(data):
    wins = [];
    for i in range(len(data)):
        if data[i] >= 0:
            wins.append(data[i]);
    avg = sma(wins, len(wins));
    return avg[0];

def avgloss(data):
    loss = [];
    for i in range(len(data)):
        if data[i] < 0: loss.append(data[i]);
    avg = sma(loss, len(loss));
    return avg[0];

def kelly(data):
    exp = er(data) + 1
    winr = winratio(data);
    return winr - (1-winr) / exp;

def se(data, size=0):
    if size == 0:
        size = len(data);
    stdv = std(data);
    return stdv / (size ** 0.5)

def zscore(data, l):
    out = []; pl = data[0:l-1];
    for i in range(l-1, len(data)):
        pl.append(data[i]);
        mean = sma(pl, l);
        stdv = std(pl, l);
        out.append((data[i]-mean[0])/stdv);
        pl = pl[1:];
    return out;

def normalize_pair(data1, data2):
    f = (data1[0] + data2[0]) / 2; ret = [[f,f]];
    for i in range(1, len(data1)):
        ret.append([ret[len(ret)-1][0]*((data1[i]-data1[i-1])/data1[i-1]+1),ret[len(ret)-1][1]*((data2[i]-data2[i-1])/data2[i-1]+1)]);
    return ret;

def normalize_from(data, value):
    ret = [value];
    for i in range(1, len(data)):
        ret.append(ret[len(ret)-1]*((data[i]-data[i-1])/data[i-1]+1));
    return ret;

def fisher(data, l):
    out = []; fish = 0; v1 = 0;
    for i in range(l, len(data)+1):
        pl = data[i-l:i]; pf = fish;
        mn = min(pl);
        v1 = .33*2*((data[i-1]-mn)/(max(pl)-mn)-.5)+.67*v1;
        if v1 > .99:
            v1 = .999;
        if v1 < -.99:
            v1 = -.999;
        fish = 0.5 * math.log((1+v1)/(1-v1)) + 0.5 * pf;
        out.append([fish, pf]);
    return out[1:];

def cross(d1, d2):
    d1 = d1[len(d1)-len(d2):];
    cross = (d1[0] > d2[0]);
    indexes = [];
    for i in range(len(d1)):
        if d1[i] < d2[i] and cross:
            indexes.append({"index": i, "cross": False});
            cross = False;
        if d1[i] > d2[i] and cross == False:
            indexes.append({"index": i, "cross": True});
            cross = True;
    return indexes;

def halftrend(data, atrlen, amplitude, deviation):
    out = []; nexttrend = [0]; trend = [0]; up = [0]; down = [0]; direction = None;
    for i in range(atrlen, len(data)+1):
        pl = data[i-atrlen:i];
        maxlow = pl[len(pl)-2][2];
        minhigh = pl[len(pl)-2][0];
        atr2 = atr(pl, atrlen);
        atr2 = atr2[len(atr2)-1] / 2;
        dev = deviation * atr2;
        highprice = max(pl[len(pl)-2][0], pl[len(pl)-1][0]);
        lowprice = min(pl[len(pl)-2][2], pl[len(pl)-1][2]);
        highs = list(map(lambda x: x[0], pl[len(pl)-amplitude:len(pl)]));
        lows = list(map(lambda x: x[2], pl[len(pl)-amplitude:len(pl)]));
        highma = sma(highs, amplitude);
        lowma = sma(lows, amplitude);
        if nexttrend[len(nexttrend)-1] == 1:
            maxlow = max(lowprice, maxlow);
            if highma[0] < maxlow and pl[len(pl)-1][1] < pl[len(pl)-2][2]:
                trend.append(1);
                nexttrend.append(0);
                minhigh = pl[len(pl)-2][0]
        else:
            minhigh = min(highprice, minhigh);
            if lowma[0] > minhigh and pl[len(pl)-1][1] < pl[len(pl)-2][0]:
                trend.append(0);
                nexttrend.append(1);
                maxlow = lowprice
        if trend[len(trend)-1] == 0:
            if not math.isnan(trend[len(trend)-2]) and trend[len(trend)-2] != 0:
                if math.isnan(down[len(down)-2]):
                    up.append(down[len(down-1)]);
                else:
                    up.append(down[len(down)-2]);
            else:
                if math.isnan(up[len(up)-2]):
                    up.append(maxlow);
                else:
                    up.append(max(up[len(up)-2], maxlow));
            direction = 'long';
            atrHigh = up[len(up)-1] + dev;
            atrLow = up[len(up)-1] - dev;
        else:
            if not math.isnan(trend[len(trend)-2] and trend[len(trend)-2] != 1):
                if math.isnan(up[len(up)-2]):
                    down.append(up[len(up)-1]);
                else:
                    down.append(up[len(up)-2]);
            else:
                if math.isnan(down[len(down)-2]):
                    down.append(minhigh);
                else:
                    down.append(min(minhigh, down[len(down)-2]));
            direction = 'short';
            atrHigh = down[len(down)-1] + dev;
            atrLow = down[len(down)-1] - dev;
        if trend[len(trend)-1] == 0:
            out.append([atrHigh, up[len(up)-1], atrLow, direction]);
        else:
            out.append([atrHigh, down[len(down)-1], atrLow, direction]);
    return out

def log(d):
    return list(map(lambda x: math.log(x),d));

def exp(d):
    return list(map(lambda x: math.exp(x),d));

def covariance(data1, data2, length):
    out = []; x_mean = sma(data1, len(data1));
    y_mean = sma(data2, len(data2)); res = 0;
    for z in range(length, len(data1)+1):
        x = data1[z-length:z]; y = data2[z-length:z];
        x_mean = sma(x, length); y_mean = sma(y, length);
        for i in range(length):
            res += (x[i]-x_mean[0])*(y[i]-y_mean[0]);
        out.append(res/length);
    return out;

def ncdf(x, mean=0, std=0):
    if mean != 0 and std != 0:
        x = (x - mean) / std;
    t = 1 / ( 1+ .2315419 * abs(x));
    d = .3989423*math.exp(-x*x/2);
    p = d*t*(.3193815+t*(-.3565638+t*(1.781478+t*(-1.821256+t*1.330274))));
    if x > 0:
        return 1 - p;
    else:
        return p;
    
def zigzag(data, perc=0.05):
    indexes = []; min = float('inf'); max = float('-inf');
    lmin = False; lmax = False;
    if isinstance(data[0], list):
        for i in range(len(data)):
            if lmin:
                if min >= data[i][1]: min = data[i][1];
                if indexes[len(indexes)-1]['value'] >= data[i][1]:
                    indexes[len(indexes)-1]['value'] = data[i][1];
                    indexes[len(indexes)-1]['index'] = i;
                    continue
                try:
                    hdif = (data[i][0]-min)/min;
                except:
                    hdif = 100
                if hdif > perc:
                    indexes.append({'index': i, 'value': data[i][0]});
                    lmax = True;
                    lmin = False;
                    min = float('inf');
            elif lmax:
                if max <= data[i][1]: max = data[i][1];
                if indexes[len(indexes)-1]['value'] <= data[i][0]:
                    indexes[len(indexes)-1]['value'] = data[i][0];
                    indexes[len(indexes)-1]['index'] = i;
                    continue
                try:
                    ldif = (max-data[i][1])/data[i][1];
                except:
                    ldif = 100
                if ldif > perc:
                    indexes.append({'index': i, 'value': data[i][1]});
                    lmin = True;
                    lmax = False;
                    max = float('-inf');
            else:
                if min >= data[i][0]: min = data[i][1];
                if max <= data[i][0]: max = data[i][0];
                if i == 0: continue
                try:
                    hdif = (data[i][0]-min)/min;
                except:
                    hdif = 100
                try:
                    ldif = (max-data[i][1])/max;
                except:
                    ldif = 100
                if ldif > perc and hdif < perc:
                    lmin = True;
                    indexes.append({'index': 0, 'value': data[0][0]});
                    indexes.append({'index': i, 'value': data[i][1]});
                elif hdif > perc and ldif < perc:
                    lmax = True;
                    indexes.append({'index': 0, 'value': data[0][1]});
                    indexes.append({'index': i, 'value': data[i][0]});
                else:
                    if ldif > hdif:
                        lmin = True;
                        indexes.append({'index': 0, 'value': data[0][0]});
                        indexes.append({'index': i, 'value': data[i][1]});
                    else:
                        lmax = True;
                        indexes.append({'index': 0, 'value': data[0][1]});
                        indexes.append({'index': i, 'value': data[i][0]});
    else:
        for i in range(len(data)):
            if lmin:
                if min >= data[i]: min = data[i];
                if indexes[len(indexes)-1]['value'] >= data[i]:
                    indexes[len(indexes)-1]['value'] = data[i];
                    indexes[len(indexes)-1]['index'] = i;
                    continue
                try:
                    hdif = (data[i]-min)/min;
                except:
                    hdif = 100
                if hdif > perc:
                    indexes.append({'index': i, 'value': data[i]});
                    lmax = True;
                    lmin = False;
                    min = float('inf');
            elif lmax:
                if max <= data[i]: max = data[i];
                if indexes[len(indexes)-1]['value'] <= data[i]:
                    indexes[len(indexes)-1]['value'] = data[i];
                    indexes[len(indexes)-1]['index'] = i;
                    continue
                ldif = (max-data[i])/data[i];
                if ldif > perc:
                    indexes.append({'index': i, 'value': data[i]});
                    lmin = True;
                    lmax = False;
                    max = float('-inf');
            else:
                if min >= data[i]: min = data[i];
                if max <= data[i]: max = data[i];
                if i == 0: continue
                try:
                    hdif = (data[i]-min)/min;
                except:
                    hdif = 100
                try:
                    ldif = (max-data[i])/max;
                except:
                    ldif = 100
                indexes.append({'index': 0, 'value': data[0]});
                indexes.append({'index': i, 'value': data[i]});
                if ldif > perc and hdif < perc:
                    lmin = True;
                elif hdif > perc and ldif < perc:
                    lmax = True;
                else:
                    if ldif > hdif:
                        lmin = True;
                    else:
                        lmax = True;
    final = [indexes[0]['value']];
    for i in range(1,len(indexes)):
        length = indexes[i]['index'] - indexes[i-1]['index'];
        delta = (indexes[i]['value'] - indexes[i-1]['value']) / length;
        for x in range(1, length+1):
            final.append(x*delta+indexes[i-1]['value']);
    return final;

def psar(data, step=0.02, maxi=0.2):
    furthest = data[0]; up = True; accel = step; prev = data[0];
    sar = data[0][1]; extreme = data[0][0]; final = [sar];
    for i in range(1, len(data)):
        sar = sar + accel * (extreme - sar);
        if up:
            sar = min(sar, furthest[1], prev[1]);
            if data[i][0] > extreme:
                extreme = data[i][0];
                accel = min(accel+step, maxi);
        else:
            sar = max(sar, furthest[0], prev[0]);
            if data[i][1] < extreme:
                extreme = data[i][0];
                accel = min(accel + step, maxi);
        if (up and data[i][1] < sar) or (not up and data[i][0] > sar):
            accel = step;
            sar = extreme;
            up = not up;
            if not up:
                extreme = data[i][1];
            else:
                extreme = data[i][0];
        furthest = prev;
        prev = data[i];
        final.append(sar);
    return final;

def fibbands(data, length=20, deviations=3):
    pl = []; deviation = []; ma = vwma(data, length); boll = [];
    for i in range(0, len(data)):
        pl.append(data[i][0]);
        if len(pl) >= length:
            devi = std(pl, length);
            deviation.append(devi*deviations);
            pl = pl[1:];
    for i in range(0, len(ma)):
        upper1 = ma[i] + (0.236 * deviation[i]);
        upper2 = ma[i] + (0.382 * deviation[i]);
        upper3 = ma[i] + (0.5 * deviation[i]);
        upper4 = ma[i] + (0.618 * deviation[i]);
        upper5 = ma[i] + (0.764 * deviation[i]);
        upper6 = ma[i] + deviation[i];
        lower1 = ma[i] - 0.236 * deviation[i];
        lower2 = ma[i] - 0.382 * deviation[i];
        lower3 = ma[i] - 0.5 * deviation[i];
        lower4 = ma[i] - 0.618 * deviation[i];
        lower5 = ma[i] - 0.764 * deviation[i];
        lower6 = ma[i] - deviation[i];
        boll.append([upper6, upper5, upper4, upper3, upper2, upper1, ma[i], lower1, lower2, lower3, lower4, lower5, lower6])
    return boll;

def supertrend(data, length=20, multiplier=3):
    at = atr(data, length); trend = [];
    for i in range(length-1, len(data)):
        trend.append([(data[i][0] + data[i][2]) / 2 + multiplier * at[i], (data[i][0] + data[i][2]) / 2 - multiplier * at[i]]);
    return trend

def cwma(data, weights):
    ma = [];
    for i in range(len(weights),len(data)+1):
        pl = data[i-len(weights):i]; sum = 0; weight = 0;
        for q in range(len(weights)):
            sum += pl[q] * weights[q];
            weight += weights[q];
        ma.append(sum / weight);
    return ma;

fibnumbers = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181];
def permutations(data):
    total = 1;
    for i in range(len(data)):
        total *= data[i];
    return total;
def mse(data1, data2):
    err = 0;
    for i in range(len(data1)):
        err += math.pow((data2[i]-data1[i]), 2);
    return err / len(data1);
def cum(data, length):
    res = [];
    for i in range(length, len(data)+1):
        res.append(sum(data[i-length:i]));
    return res;
def vwwma(d, length=20):
    data = []; weight = 0; vwm = [];
    for i in range(len(d)):
        data.append([d[i][0]*d[i][1],d[i][1]]);
    for i in range(1, length+1):
        weight += i;
    for i in range(length, len(data)+1):
        pl = data[i-length:i];
        totalv = 0; totalp = 0;
        for q in range(len(pl)):
            totalv += pl[q][1] * (q+1) / weight;
            totalp += pl[q][0] * (q+1) / weight;
        vwm.append(totalp / totalv);
    return vwm;
def elderray(data, length=13):
    eld = [];
    for i in range(length, len(data)+1):
        pl = data[i-length:i];
        low = min(pl);
        high = max(pl);
        em = ema(pl, len(pl))
        eld.append([high-em[0],low-em[0]]);
    return eld;
def hv(data, length=20):
    hv = [];
    for i in range(length, len(data)+1):
        ss = ssd(data[i-length:i]);
        vari = ss[0] / length;
        hv.append(math.sqrt(vari));
    return hv;
def rvi(data, l=20):
    num = []; dnom = []; rv = [];
    for i in range(3, len(data)):
        num.append((data[i][3]-data[i][0]+2*(data[i][3]-data[i-1][0])+2*(data[i][3]-data[i-2][0])+(data[i][3]-data[i-3][0]))/6);
        dnom.append((data[i][1]-data[i][2]+2*(data[i][1]-data[i-1][2])+2*(data[i][1]-data[i-2][2])+(data[i][1]-data[i-3][2]))/6);
        if len(num) >= l:
            sn = sma(num, l);
            dn = sma(dnom, l);
            rv.append(sn[0]/dn[0]);
            num = num[1:len(num)];
            dnom = dnom[1:len(dnom)];
    return rv;
def rvi_signal(rv):
    sig = [];
    for i in range(3,len(rv)):
        sig.append((rv[i]+2*rv[i-1]+2*rv[i-2]+rv[i-3])/6);
    return sig;
def rsi_divergence(data, length, rs=wrsi):
    rd = rs(data, length); out = [];
    data = mom(data[length-1:], 2);
    for i in range(0, len(data)):
        if (data[i] > 0 and rd[i] < 0) or (data[i] < 0 and rd[i] > 0):
            out.append(1);
        else:
            out.append(0);
    return out;
def divergence(data1, data2):
    if len(data1) > len(data2):
        data1 = data1[len(data1)-len(data2):];
    if len(data2) > len(data1):
        data2 = data2[len(data2)-len(data1):];
    out = [];
    for i in range(1, len(data1)):
        if (data1[i] > data1[i-1] and data2[i] < data2[i-1]) or (data1[i] < data1[i-1] and data2[i] > data2[i-1]):
            out.append(1);
        else:
            out.append(0);
    return out;

def times_up(data, l):
    out = [];
    for i in range(l, len(data)):
        up = 1;
        for x in range(i-l+1, i+1):
            if data[x-1] > data[x]:
                up = 0;
                break;
        out.append(up);
    return out;

def times_down(data, l):
    out = [];
    for i in range(l, len(data)):
        dn = 1;
        for x in range(i-l+1, i+1):
            if data[x-1] < data[x]:
                dn = 0;
                break;
        out.append(dn);
    return out;

class Binom:
    
    def __init__(self):
        self.N = N
        self.k = k
        self.p = p
    
    def BinomialCoefficient(N,k):
        a = math.factorial(N)
        b = math.factorial(N-k)
        c = math.factorial(k)
        return a/(b*c)
    
    def binomialDensity(N,p,k):
        """
        """
        a = p**k
        b = (1-p)**(N-k)
        f =  BinomialCoefficient(N,k)*a*b
        return f 
    

def Gamma(a):
    return math.gamma(a)

import math

class binom():
    "test"

    def __init__(self,N = 10, k = 1, p = 0.5):
        """
        test
        """
        self.N = N
        self.k = k
        self.p = p
        self.q = 1-self.p

        self.BinomialCoefficient(self.N, self.k)

    def BinomialCoefficient(self,N = None,k = None):
        """
        
        """
        if N == None:
            N = self.N
        if k == None:
            k = self.k
        
        a = math.factorial(N)
        b = math.factorial(N-k)
        c = math.factorial(k)
        self.BinomCoef = a/(b*c)
        return(self.BinomCoef)

    def BinomialDensity(self, N = None,k = None, p = None):
        """
        
        """
        if N == None:
            N = self.N
        if p == None:
            p = self.p
            q = self.q
        if k == None:
            k = self.k
            
        q = (1-p) 
        a = p**k
        b = q**(N-k)
        f =  self.BinomialCoefficient(N, k)*a*b
        return(f) 
    
Bin = binom()
Bin.BinomialDensity(10, 5, 0.5)
help(binom)

class Poiss:
    
    def __init__(self):
        self.lam = 0
        self.k = 0

    def PoissonDensity(lam,k):
        """
        """
        a = np.exp(-lam)
        b = np.power(lam,k)
        c = math.factorial(k)
        f = np.divide(a*b,c)
        return f

class normal:
    
    def __init__(self):
        self.mu = 0
        self.sigma = 0
        
    def GaussianDensity(z,mu,sigma):
        num = z-mu    
        den = 2*(sigma**2)
        K = np.reciprocal(np.sqrt(2*math.pi)*sigma)
        f = K*np.exp(-np.divide(num**2,den))
        return f        


def fOrnUhlenSol(t,x,a,s):
    #probability density for Ornstein-Uhlenbeck
    #t is at time t, x is state x, a is the return to long term mean, s is the spread
    #https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process Fokker-planck equation representation
    
    D = s**2/2
    return np.sqrt(a/(2*np.pi*D*(1-np.exp(-2*a*t)))) * np.exp(-0.5*a/D * (x-x0*np.exp(-a*t))**2 / (1-np.exp(-2*a*t)) )

def fOrnUhlenSol_ss(x,a,s):
    #dunno
    return np.sqrt(a/np.pi/s**2) * np.exp(-a*x**2/s**2)

def fWienerSol(t,x):
    #Probability density for Wiener process
    return 1/sqrt(2*pi*t) * exp(-0.5*x**2/t)


def PoissonDensity(lam,k):
    a = np.exp(-lam)
    b = np.power(lam,k)
    c = math.factorial(k)
    f = a*b/c
    return f

def LogitDensity(z,mu,sigma):
    num = np.log(z/(1-z))-mu    
    den = 2*(sigma**2)
    K = np.reciprocal(sigma*np.sqrt(2*math.pi)*z*(1-z))
    f = K*np.exp(-np.divide(num**2,den))
    return f

def ProbitDensity(z,mu,sigma):
    pzInverse = np.divide(norm.ppf(z)-mu,sigma)
    num = gaussianDensity(pzInverse,0,1)
    den = sigma*gaussianDensity(norm.ppf(z),0,1)
    f = np.divide(num,den)
    return f

def GaussianDensity(z,mu,sigma):
    num = z-mu    
    den = 2*(sigma**2)
    K = np.reciprocal(np.sqrt(2*math.pi)*sigma)
    f = K*np.exp(-np.divide(num**2,den))
    return f        

#def logGaussianDensity(z,mu,sigma):
#    num = np.log(z)-mu    
#    den = 2*(sigma**2)
#    K = np.reciprocal(np.multiply(np.sqrt(2*math.pi)*sigma,z))
#    f = K*np.exp(-np.divide(num**2,den))
#    return f        

def tDensity(z,mu,sigma,nu):
    g1 = math.gamma((nu+1)/2)
    g2 = math.gamma(nu/2)
    K = np.divide(g1,g2*np.sqrt(nu*math.pi)*sigma)
    power = np.divide((z-mu)**2,nu*(sigma**2))
    f = K*np.power(1+power,-(nu+1)/2)
    return f       

def chi2Density(z,nu):
    g1 = math.gamma(nu/2)
    constant = np.multiply(g1,2**(nu/2))
    term1 = np.power(z,(nu/2)-1)
    term2 = np.exp(-z/2)   
    f = np.reciprocal(constant)*term1*term2
    return f       

def gigDensity(x,myA):
    constant = np.divide(1,2*scipy.special.kn(1, myA))
    f = constant*np.exp(-0.5*myA*(x+1/x))
    return f

def betaDensity(z,a,b):
    term1 = np.power(z,a-1)
    term2 = np.power(1-z,b-1)   
    f = np.reciprocal(computeBeta(a,b))*term1*term2
    return f           

def asrfDensity(x,p,rho):
    a = np.sqrt(np.divide(1-rho,rho))
    b = np.power(np.sqrt(1-rho)*norm.ppf(x)-norm.ppf(p),2)
    c = 0.5*(np.power(norm.ppf(x),2) - b/rho)
    return a*np.exp(c)          

def gammaDensity(z,a,b):
    constant = np.divide(b**a,math.gamma(a))
    t1 = np.exp(-b*z)
    t2 = np.power(z,a-1)
    pdf =  constant*t1*t2
    return pdf            

def logNormalDensity(z,mu,sigma):
    constant = np.divide(1,z*sigma*np.sqrt(2*math.pi))
    num = -(np.log(z)-mu)**2
    den = 2*sigma**2
    pdf =  constant*np.exp(num/den)
    return pdf   

#def weibullDensity1(z,a,b):
#    constant = np.divide(a,b**a)
#    t1 = np.power(z/b,a-1)
#    t2 = np.exp(-(z/b)**a)
#    pdf =  constant*t1*t2
#    return pdf   

def weibullDensity(x,a,b):
     return (a/b)*(x/b)**(a-1)*np.exp(-(x/b)**a)
    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sqlalchemy as db
from sqlalchemy import create_engine, text


class claim_size_dist:
    
    def __init__(self, data, claim_size_dist = "pareto"):
        self.data = data.data.copy()
        self.data['claim size'] = np.float64(0)
        
        if claim_size_dist == "pareto":
            self.data['claim size'] = (np.random.pareto(1, len(self.data)) + 1)*30
        elif claim_size_dist == "normal":
            self.data['claim size'] = (np.random.normal(1,0.5, len(self.data)))


class savings:
    
    def __init__(self, s0 = 100, t = 15, i = None, typ = "bank", gender = "male", age = 55, location = "dk"):
        
        self.typ = typ
        self.p = 0.9975
        self.t = t
        self.gender = gender
        if self.gender.lower() == "male" or self.gender.lower() == "men":
            self.gender = "Men"
        elif self.gender.lower() == "female" or self.gender.lower() == "women":
            self.gender = "Women"
        self.age = age
        self.location = location
        
        
        self.datb = database()
        self.interest = self.datb.interest_rates
        if self.location.lower() == "dk":
            self.interest = self.interest[['interest rate dk']]
        elif self.location.lower() == "uk":
            self.interest = self.interest[['interest rate uk']]
        elif self.location.lower() == "us":
            self.interest = self.interest[['interest rate us']]
        
        self.interests = []
        for j in range(0, self.t):
            if i == None:
                mean_rate = np.mean(self.interest[j*12:(j+1)*12])
            else:
                mean_rate = i
            self.interests.append(mean_rate)
            
        
        self.savings =  pd.DataFrame(np.repeat(np.float64(s0),self.t+1), columns = ['Savings'])
        for j in range(1, self.t+1):
                self.savings.iloc[j,0] = self.savings.iloc[j-1,0] * (1+(self.interests[j-1]/100))
                
        if self.typ.lower() == "insurance":
            
            self.life = self.datb.life_table
            self.life = self.life[self.life['gender'] == self.gender]
            self.life = self.life[self.life['age'] >= self.age]
            self.life = self.life.head(self.t+1)
            self.life = self.life.reset_index(drop = True)
            
            
            self.savings['Probability'] = self.life['p']
            self.savings['Mortality savings'] = np.repeat(np.float64(s0),self.t+1)
            for j in range(1, self.t+1):
                self.savings.iloc[j,1] = self.savings.iloc[j,1] * self.savings.iloc[j-1,1]
                self.savings.iloc[j,2] = self.savings.iloc[j,0] / self.savings.iloc[j,1]
        
        
S = savings(1000, 50, 2, "insurance", "male", 25)



class claim_number_process:
    
    def __init__(self, time, process):
        
        if process == "poisson":
            self.data = poisson(time, lambd = 1).data
        
        #elif process == "pareto":
            #self.data = pareto(time, a = 1, m = 50).data
            
class poisson:
    
    def __init__(self,time,lambd):
        t = 0
        i = 0
        lambd = 0.5 # sandsynlighed for nyt claim
        event = []
        while t <= time:

            t = t + random.expovariate(lambd)
            i = i + 1
            event.append([t, i])

        self.data = pd.DataFrame(event, columns = ["claim times", "claim number"])
        self.data.set_index("claim times", inplace=True)
        
#high interest rate + other factors = low inflation?
#low interest rate + other factors = high inflation?
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go



years = 5
timeframe = 12*years

participants = 100
categories = 15

DWI = pd.DataFrame(np.repeat(2,participants))
CPI = pd.DataFrame(np.repeat(2,categories))


#



DWI_M = []
CPI_M = []
REALLON = []

# each participant in an economy has normal dist 
for i in range(1, timeframe):
    
    DWI[i] = DWI[i-1] + (abs(np.random.normal(loc=0, scale =0.00407, size=participants)))
    CPI[i] = CPI[i-1] + np.random.normal(loc=0, scale =0.005, size=categories)
    
for i in range(0,timeframe):
    DWI_M.append(round(np.mean(DWI[i]),6))
    CPI_M.append(round(np.mean(CPI[i]),6))

for i in range(0, timeframe):
    REALLON.append(DWI_M[i]-CPI_M[i])
display(DWI)
display(CPI)
T = pd.DataFrame([CPI_M,DWI_M,REALLON], index = ["CPI", "DWI", "REALLON"])
display(T)



plt.bar(T.columns, list(T.loc["CPI"]))
plt.axhline(y=0, color='k', linestyle='-')
#plt.ylim(-0.05, 0.05)
plt.title("CPI")
plt.xticks([0,12,24,36,48,60], ['0år','1år', '2år', '3år','4år','5år'])
plt.show()

plt.bar(T.columns, list(T.loc["DWI"]))
plt.axhline(y=0, color='k', linestyle='-')
#plt.ylim(-0.05, 0.05)
plt.title("DWI")
plt.xticks([0,12,24,36,48,60], ['0år','1år', '2år', '3år','4år','5år'])
plt.show()

plt.bar(T.columns, list(T.loc["REALLON"]))
plt.axhline(y=0, color='k', linestyle='-')
#plt.ylim(-0.05, 0.05)
plt.title("REALLON")
plt.xticks([0,12,24,36,48,60], ['0år','1år', '2år', '3år','4år','5år'])
plt.show()

In [ ]:
"""
TMRW.FINANCE

Created on Mon Apr 15 22:44:02 2024

Version 1.0.0 bundled on Fri Aug 09 12:00:00 2024

@Author: Mark Daniel Balle Brezina
@Contributors: Magnus Faber, Jonathan Emil Balle Brezina
"""
import yfinance as yf
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt

from datetime import datetime, date, timedelta, timezone
import sqlalchemy as sa

from scipy.stats import norm
from scipy.stats import linregress
from scipy import integrate
from statsmodels.tsa.stattools import adfuller
from hmmlearn.hmm import GaussianHMM

# -*- coding: utf-8 -*-
"""
Created on Wed Nov  1 23:23:08 2023

@author: Markb

    Created on 09-Apr-2022
    Authors: Kiril Zelenkovski, Petar Jolakoski, Pece Trajanovski

    Subordinator for calculation of CTRWs (O-U process)

"""

## This is an implementation of a simple integrator for a generic
# jump-diffusion process. This includes for a Euler─Mayurama and a Milstein
# integration scheme. To use the Milstein scheme the derivative of the diffusion
# function needs to be given. Created by Leonardo Rydin Gorjão and Pedro G. Lind




import pandas as pd
from IPython.display import Markdown, display
import numpy as np


class SSP:
    """
    The SSP class of stochastic processes
    """
    def __init__(self, runs = 1, timeframe = 30):
        """
        The SSP class takes in number of runs and timeframe to store it for later use with stochastic processes
        """
        self.runs = runs
        self.timeframe = timeframe

    #Random walk
    
    def RW(self, _print_ = False):
        """
        From the SSP class, the number of runs and the timeframe has been indicated.
        By adding True or _print_ = True in the input, the user will have a printout 
        of the mathematical description for the process. For this stochastic process
        no additional information is needed and the output will be a dataframe with
        number of runs times timeframe
        
        Abbreviations
        -------------
        The function is synonomous with RandomWalk, randomwalk, Random_Walk, R_W, r_w, rw and RW
        
        Parameters
        ----------
        _print_: (True/False): True prints the mathematical description of the formula used in the simulation
        
        Returns
        -------
        A numpy array of simulated paths for the random walk process
        
        Examples
        --------
        >>> random_walk(False)
        array([[ 0], [ 1], [ 2]],[[ 0], [ -1], [ 0]])
        
        >>> random_walk(True)
        This stochastic process is the discrete random walk, with the equation:  𝑥𝑡=𝑥𝑡−1+𝜖𝑡, where 𝜖𝑡 is the Rademacher distribution at time t:  𝑓(𝑘)={1/21/2𝑖𝑓𝑘=1𝑖𝑓𝑘=−1
        array([[ 0], [ 1], [ 2]],[[ 0], [ -1], [ 0]])
        
        """
        if _print_ == True:
            equation = r'$x_t = x_{t-1} + \epsilon_t$'
            eq = r'$ f(k)=\left\{\begin{matrix}1/2 & if \;\;\; k=1 \\1/2 & if \;\;\; k=-1 \\\end{matrix}\right.$'
            display(Markdown((f'This stochastic process is the discrete random walk, with the equation: {equation}, where $\epsilon_t$ is the Rademacher distribution at time t: {eq}')))

        all_walks = []
        for i in range(self.runs) :
            random_walk = [0]
            for t in range(self.timeframe) :
                step = random_walk[-1]
                probability = np.random.randint(0,2)
                if probability == 0:
                    step = step + 1
                elif probability == 1:
                    step = step - 1
                random_walk.append(step)
            all_walks.append(random_walk)
        dataframe_allwalks = np.array(all_walks)
        walks = pd.DataFrame(dataframe_allwalks)
        return(walks)
    
    def randomwalk(self, _print_ = False):
        """
        This is a synonumous function to random_walk
        """
        return self.RW(_print_)
    
    def RandomWalk(self, _print_ = False):
        """
        This is a synonumous function to random_walk
        """
        return self.RW(_print_)
    
    def Random_Walk(self, _print_ = False):
        """
        This is a synonumous function to random_walk
        """
        return self.RW(_print_)
    
    def R_W(self, _print_ = False):
        """
        This is a synonumous function to random_walk
        """
        return self.RW(_print_)  
    
    def r_w(self, _print_ = False):
        """
        This is a synonumous function to random_walk
        """
        return self.RW(_print_)
    
    def rw(self, _print_ = False):
        """
        This is a synonumous function to random_walk
        """
        return self.RW(_print_)
    
    def random_walk(self, _print_ = False):
        """
        This is a synonumous function to random_walk
        """
        return self.RW(_print_)  
    
    #Wiener process 
    
    def WP(self, _print_ = False):
        """
       From the SSP class, the number of runs and the timeframe has been indicated.
       By adding True or _print_ = True in the input, the user will have a printout 
       of the mathematical description for the process. For this stochastic process
       no additional information is needed and the output will be a dataframe with
       number of runs times timeframe
       
       Abbreviations
       -------------
       The function is synonomous with WienerProcess, wienerprocess, Wiener_Process, W_P, w_p, wp and WP.
       
       Parameters
       ----------
       _print_: (True/False): True prints the mathematical description of the formula used in the simulation
       
       Returns
       -------
       A numpy array of simulated paths for the simple wiener process
       
       Examples
       --------
       >>> wiener_process(False)
       array([[ 0], [ 1], [ 2]],[[ 0], [ -1], [ 0]])
       
       >>> wiener_process(True)
       This stochastic process is the discrete wiener process, with the equation: 𝑥𝑡=𝑥𝑡−1+√dt⋅𝑁𝑑𝑖𝑠𝑡, where 𝑁𝑑𝑖𝑠𝑡 is the standard normal distribution and √d𝑡 is the squareroot of the time difference between steps
       array([[ 0], [ 1], [ 2]],[[ 0], [ -1], [ 0]])
       
        """
        if _print_ == True:
            equation = r'$x_t = x_{t-1} + \sqrt{dt} \cdot N_{dist}$'
            display(Markdown((f'This stochastic process is the discrete wiener process, with the equation: {equation}, where' +r'${N_{dist}}$'+' is the standard normal distribution and $\sqrt dt$ is the squareroot of the time difference between steps')))
       
        all_walks = []
        for j in range(self.runs):
            walk = np.zeros((self.timeframe))
            walk[0] = 1
            for i in range(1, self.timeframe):
                yi = np.random.normal()
                walk[i] = walk[i-1]+(yi)*np.sqrt(1/self.timeframe)
            all_walks.append(walk)
        dataframe_allwalks = np.array(all_walks)
        walks = pd.DataFrame(dataframe_allwalks)

        return(walks)
   
    def wp(self, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.WP(_print_)

    def wiener_process(self, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.WP(_print_) 

    def w_p(self, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.WP(_print_) 

    def W_P(self, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.WP(_print_) 

    def Wiener_Process(self, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.WP(_print_) 

    def wienerprocess(self, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.WP(_print_)

    def WienerProcess(self, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.WP(_print_)
    
    
    #Brownian motion
    
    def BM(self, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.WP(_print_)
    
    def bm(self, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.WP(_print_)
    
    def B_M(self, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.WP(_print_)
    
    def b_m(self, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.WP(_print_) 
   
    def brownianmotion(self, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.WP(_print_) 
   
    def BrownianMotion(self, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.WP(_print_) 
   
    def Brownian_Motion(self, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.WP(_print_) 
   
    def brownian_motion(self, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.WP(_print_) 
   
    
    #Arithmetic brownian motion
    def ABM(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        From the SSP class, the number of runs and the timeframe has been indicated.
        By adding True or _print_ = True in the input, the user will have a printout 
        of the mathematical description for the process. For this stochastic process
        no additional information is needed and the output will be a dataframe with
        number of runs times timeframe
        
        Abbreviations
        -------------
        The function is synonomous with WienerProcess, wienerprocess, Wiener_Process, W_P, w_p, wp and WP.
        
        Parameters
        ----------
        _print_: (True/False): True prints the mathematical description of the formula used in the simulation
        
        Returns
        -------
        A numpy array of simulated paths for the simple wiener process
        
        Examples
        --------
        >>> wiener_process(False)
        array([[ 0], [ 1], [ 2]],[[ 0], [ -1], [ 0]])
        
        >>> wiener_process(True)
        This stochastic process is the discrete wiener process, with the equation: 𝑥𝑡=𝑥𝑡−1+√dt⋅𝑁𝑑𝑖𝑠𝑡, where 𝑁𝑑𝑖𝑠𝑡 is the standard normal distribution and √d𝑡 is the squareroot of the time difference between steps
        array([[ 0], [ 1], [ 2]],[[ 0], [ -1], [ 0]])
        
        """
        if _print_ == True:
            equation = r'$x_t = x_{t-1} + \mu \cdot dt + \sigma \cdot \sqrt{dt} \cdot N_{dist}$'
            display(Markdown((f'This stochastic process is the discrete wiener process, with the equation: {equation}, where' +r'${N_{dist}}$'+' is the standard normal distribution and $\sqrt dt$ is the squareroot of the time difference between steps')))
        
        dt = 1/self.timeframe
        dW = np.sqrt(dt)*np.random.randn(self.runs, self.timeframe)
        dS = mu*dt + sigma*dW
        
        dS = np.insert(dS, 0, 1, axis=1)
        walks = np.cumsum(dS, axis=1)
        walks = pd.DataFrame(walks)
        return(walks)
    
    def abm(self, mu = 1, sigma = 0.25, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.ABM(mu, sigma, _print_) 
   
    def a_b_m(self, mu = 1, sigma = 0.25, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.ABM(mu, sigma, _print_) 
   
    def A_B_M(self, mu = 1, sigma = 0.25, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.ABM(mu, sigma, _print_)  
    
    def ArithmeticBrownianMotion(self, mu = 1, sigma = 0.25, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.ABM(mu, sigma, _print_) 
   
    def arithmeticbrownianmotion(self, mu = 1, sigma = 0.25, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.ABM(mu, sigma, _print_) 
   
    def arithmetic_brownian_motion(self, mu = 1, sigma = 0.25, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.ABM(mu, sigma, _print_) 
   
    def Arithmetic_Brownian_Motion(self, mu = 1, sigma = 0.25, _print_ = False):
       """
       This is a synonumous function to wiener_process
       """
       return self.ABM(mu, sigma, _print_) 

    #Arithmetic wiener process

    def AWP(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.ABM(mu, sigma, _print_)
    
    def awp(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.ABM(mu, sigma, _print_)
    
    def A_W_P(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.ABM(mu, sigma, _print_)
    
    def a_w_p(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.ABM(mu, sigma, _print_)
    
    def arithmetic_wiener_process(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.ABM(mu, sigma, _print_)
    
    def arithmeticwienerprocess(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.ABM(mu, sigma, _print_)
    
    def Arithmetic_Wiener_Process(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.ABM(mu, sigma, _print_)
    
    def ArithmeticWienerProcess(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.ABM(mu, sigma, _print_)
    
    
    
    
    #Exponential brownian motion
    def EBM(self, mu = 1, sigma=0.25, _print_ = False):
        """
        From the SSP class, the number of runs and the timeframe has been indicated.
        By adding True or _print_ = True in the input, the user will have a printout 
        of the mathematical description for the process. For this stochastic process
        x0 is needed as additional information and the output will be a dataframe 
        with number of runs times timeframe
        :param _print_: must be True or False
        :param x0: Is set to be 1, unless changed by the user.
        :param sigma: is set to be 0.25, unless changed by the user.
        """
        
        if _print_ == True:
            equation = r'$x_t = x_{t-1} \cdot e^{\mu-0.5\cdot \sigma^{2}} \cdot dt + \sigma \cdot \sqrt{dt} \cdot N_{dist}$'
            display(Markdown((f'This stochastic process is the discrete financial wiener process with drift, with the equation: {equation} where' + r'${N_{dist}}$'+' is the standard normal distribution and $e^{r-0.5\cdot \sigma^{2}}$ is the drift w.r.t the risk-free rate and the standard deviation')))
            
        dt = 1/self.timeframe
        walks = pd.DataFrame(np.zeros((self.runs, self.timeframe)))
        walks.iloc[:,0] = 1
        for i in range(len(walks)):
            for t in range(1, self.timeframe):
                walks.iloc[i,t] = walks.iloc[i, t - 1] * np.exp((mu - 0.5*sigma ** 2) * dt + sigma * np.sqrt(dt) * np.random.randn(1, 1))

        return(walks) 
    
    def ebm(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def e_b_m(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def E_B_M(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def Exponential_Brownian_Motion(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def exponential_brownian_motion(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def ExponentialBrownianMotion(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def exponentialbrownianmotion(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    #exponential wiener process
    
    def EWP(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def ewp(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def e_w_p(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def E_W_P(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def Exponential_Wiener_Process(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def exponential_wiener_process(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def ExponentialWienerProcess(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def exponentialwienerprocess(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    
    #geometric brownian motion
    
    def GBM(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def gbm(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def g_b_m(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def G_B_M(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def Geometric_Brownian_Motion(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def geometric_brownian_motion(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def GeometricBrownianMotion(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def geometricbrownianmotion(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)

    
    
    #geometric wiener process
    
    def GWP(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def gwp(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def g_w_p(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def G_W_P(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def Geometric_Wiener_Process(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def geometric_wiener_process(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def GeometricWienerProcess(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    def geometricwienerprocess(self, mu = 1, sigma = 0.25, _print_ = False):
        """
        This is a synonumous function to wiener_process
        """
        return self.EBM(mu, sigma, _print_)
    
    
   
    # brownian bridge
    
    def Brownian_Bridge(self, alpha, beta, sigma, _print_ = False):
        """dS = ((b-X)/(T-t))*dt + sigma*dW
        Model can support useful variance reduction techniques for pricing derivative contracts using Monte-Carlo simulation, 
        such as sampling. Also used in scenario generation.
        Requires numpy, pandas and plotly.express
        alpha = start
        beta = end
        sigma = spread
        Returns the paths, S, for the Brownian Bridge using the Euler-Maruyama method"""
        
        if _print_ == True:
            equation = r'$x_{t+1} = x_{t} + \frac{(\beta-x_t)}{(T-t+1)} + \sigma \cdot dW(t) \sqrt{dt} \cdot N_{dist}$'
            display(Markdown((f'This stochastic process is the discrete financial wiener process with drift, with the equation: {equation} where' + r'${N_{dist}}$'+' is the standard normal distribution and $e^{r-0.5\cdot \sigma^{2}}$ is the drift w.r.t the risk-free rate and the standard deviation')))
        
        dt = 1/self.timeframe
        dW = np.sqrt(dt)*np.random.randn(self.timeframe, self.runs)
        S = np.concatenate((alpha*np.ones((1, self.runs)),np.zeros((self.timeframe-1, self.runs)), beta*np.ones((1, self.runs))), axis=0)
    
        for i in range(0, self.timeframe-1):
            S[i+1,:] = S[i,:] + (beta-S[i,:])/(self.timeframe-i+1) +sigma*dW[i,:]
    
        return S
       
GBM1 = 100 * (1 + SSP(1000, 90).ABM(mu = 0.028, sigma = 0.12)-1)

mean_path1 = []
for i in range(len(GBM1.columns)):
    mean_path1.append(np.mean(GBM1.iloc[:,i]))


for i in range(len(GBM1)):
    plt.plot(GBM1.columns, GBM1.iloc[i,:], color = "lightskyblue", alpha = 0.1)
plt.plot(GBM1.columns, mean_path1, color = "k", alpha = 1.0)
plt.ylim(50,150)
plt.savefig("Brownian1")

print(mean_path1[90] - mean_path1[0])

GBM2 = 100 * (1 + SSP(1000, 90).ABM(mu = 0.028, sigma = 0.25)-1)
mean_path2 = []
for i in range(len(GBM2.columns)):
    mean_path2.append(np.mean(GBM2.iloc[:,i]))


for i in range(len(GBM2)):
    plt.plot(GBM2.columns, GBM2.iloc[i,:], color = "lightcoral", alpha = 0.1)
plt.plot(GBM2.columns, mean_path2, color = "k", alpha = 1.0)
plt.ylim(50,150)
plt.savefig("Brownian2")

print(mean_path2[90] - mean_path2[0])

lst = [[1,1],[2,1.3],[3,1.2],[4,2],[5,0.3],[6,2.1],[7,1.3],[8,2.3],[9,2.55],[10,2.61],[11,2.89],[12,3.01],[13,3.12],[14,3.52],[15,4]]
lst = pd.DataFrame(lst)

###############################################################################
#                        Slopes, returns and accelerations
#                        For calculating import 
#                        intra and inter day differences
###############################################################################

def slope(array):
    """
    """
    y = np.array(array)
    x = np.arange(len(y))
    slopes, intercept, r_value, p_value, std_err = linregress(x,y)
    return(slopes)

def returns(x, logs = False):
    """
    """
    if type(x) == list: # if you've entered a list
        _return = np.zeros(len(x))
        for i in range(len(x)):
            _return[i] = ((x[i]/x[i-1])-1)
    elif type(x) == pd.DataFrame: # if you've entered a dataframe
        _return = x.pct_change().dropna()   
    else: # if you've entered neither a list or a dataframe
        _return = x.pct_change().dropna()
    
    # if you want log returns
    if logs == True:
        _return = np.log(abs(_return))
    
    df = pd.DataFrame(data=_return, dtype=np.float64)
    df.columns = ["returns"]
    return(df)

def acceleration(x, logs = False):
    """
    """
    x = returns(x, logs)
    if type(x) == list:  # if you've entered a list
        acc = np.zeros(len(x))
        for i in range(len(x)):
            acc[i] = ((x[i]/x[i-1])-1)
    elif type(x) == pd.DataFrame: # if you've entered a dataframe
        acc = x.pct_change().dropna() 
    else: # if you've entered neither a list or a dataframe
        acc = x.pct_change().dropna()
        
     # if you want log acceleration  
    if logs == True:
        acc = np.log(abs(acc))
    
    df = pd.DataFrame(data=acc, dtype=np.float64)
    df.columns = ["acceleration"]
    return(df)

###############################################################################
#                        Moving averages and stds
#                        volume weighted averages and stds
#                        bollinger bands?
###############################################################################

# Time-weighted Moving Average
def twa(data, window = 20, typ = "sma", x = True):
    """
    time-weighted moving average
    """
    if typ == "sma": # simple moving average
        if len(data) < window+1:
            raise ValueError("not enough data")
            
        if x == True:
            
            df = []
            for i in range(len(data)):
                if i < window:
                    df.append(data[i])
                else:
                    df.append(np.mean(data[i-window:i]))

            df = pd.DataFrame(df, index = data.index)
            df = df.iloc[0:len(df), 0]
            
            return(df)
    
        elif x == False:

            df = []
            for i in range(len(data)):

                ma = np.zeros(window)

                for j in range(1,window):

                    if i < window:
                        ma[j] = data[j]


                    if i >= window:

                        ma[j] = np.mean(data[i-j:i])

                df.append(ma)

            df = pd.DataFrame(df, index = data.index)
            df = df.iloc[0:len(df), 1:len(df.columns)]
            
            return(df)
    
    elif typ == "ema": # exponential moving average

        df = pd.DataFrame(index=data.index, dtype=np.float64)
        df.loc[:,0] = data.ewm(span=window, min_periods=0, adjust=True, ignore_na=False).mean().values.flatten() # ????
    
        return(df)
      
#Time-Price Moving STD
def twstd(data, window = 20, x = True):
    """
    time-weighted moving standard deviation of prices
    also called volatility
    """
    if len(data) < window+1:
        raise ValueError("not enough data")

    if x == True:
        
        df = []
        for i in range(len(data)):
            if i < window:
                df.append(data[i])
            else:
                df.append(np.std(data[i-window:i]))

        df = pd.DataFrame(df, index = data.index)
        df = df.iloc[0:len(df), 0]
        
        return(df)
        
    elif x == False:  
        
        df = []
        for i in range(len(data)):

            mstd = np.zeros(window)

            for j in range(1,window):

                if i >= j:

                    mstd[j] = np.std(data[i-j:i])

            df.append(mstd)

        df = pd.DataFrame(df, index = data.index)
        df = df.iloc[0:len(df), 1:len(df.columns)]
    
        return(df)
   
    
# Volume-weighted Moving Average
def vwa(data, weights, window = 20):
    """
    volume-weighted moving average
    """
    
    vwa = np.zeros(window)
    vwa = list(vwa)
    for i in range(len(data)):
        
        if i < window:
            
            vwa[i] = data[i]
            
        else:
    
            v_sum = sum(weights[i-window:i])
            v_w_avg = 0
            for j in range(window):
                v_w_avg = v_w_avg + data[i-j] * (weights[i-j] / v_sum)

            vwa.append(v_w_avg)
    
    df = pd.DataFrame(vwa)
    return(df)

#Volume-weighted Moving STD
def vwstd(data, weights, window = 20):
    """
    volume-weighted moving standard deviations
    """
    
    v_w_avg = 0
    vws = np.zeros(window)
    vws = list(vws)
    for i in range(window, len(data)):
    
        v_sum = sum(weights[i-window:i])
        v_w_std = 0
        for j in range(window):
            v_w_std = v_w_std + ((data[i-j] - np.mean(data[i-window:i]))**2 )* (weights[i-j] / v_sum)

        vws.append(np.sqrt(v_w_std))
    
    df = pd.DataFrame(vws)
    return(df)


###############################################################################
#                        Technical indicators
#                        RSI, Stochastic Oscillator
#
###############################################################################

def RSI(data, window = 20):
    """
    
    """
    delta = data.diff().dropna() # Close_now - Close_yesterday
    delta = delta.reset_index(drop = True)

    u = pd.DataFrame(np.zeros(len(delta))) # make an array of 0s for the up returns
    u = u[0]
    d = u.copy() # make an array of 0s for the down returns   

    u[delta > 0] = delta[delta > 0] # for all the days where delta is up, transfer them to U
    d[delta < 0] = -delta[delta < 0] # for all the days where delta is down, transfer them to D

    u[u.index[window-1]] = np.mean( u[:window] ) #first value is sum of avg gains
    u = u.drop(u.index[:(window-1)]) #drop the days before the window opens

    d[d.index[window-1]] = np.mean( d[:window] ) #first value is sum of avg losses
    d = d.drop(d.index[:(window-1)]) #drop the days before the window opens

    RS = pd.DataFrame.ewm(u, com=window-1, adjust=False).mean() / pd.DataFrame.ewm(d, com=window-1, adjust=False).mean() # EMA(up) / EMA(down)

    RSI_ = 100 - (100 / (1 + RS))
    return(RSI_)

def FRSI(data, window = 20):
    """
    inverse fisher transform on RSI = 0.1*(rsi-50)
    fisher rsi = (np.exp(2*rsi)-1) / (np.exp(2*rsi)+1)
    """
    
    RSI_ = 0.1 * (RSI(data, window) - 50)
    F_RSI = (np.exp(2*RSI_)-1) / (np.exp(2*RSI_)+1)
    return(F_RSI)

def BB(data, window = 20, std = 2.5):
    """
    
    """
    mean_lst = np.zeros(len(data))
    std_lst = np.zeros(len(data))
    for i in range(0, window):
        mean_lst[i] = data[i]
        std_lst[i] = 0.05 * data[i]
    
    for i in range(window,len(data)):
        mean_lst[i] = np.mean(data[i-window:i])
        std_lst[i] = np.std(data[i-window:i])
        
    up = mean_lst + std * std_lst
    down = mean_lst - std * std_lst
    
    df = pd.DataFrame()
    df['Upper'] = up
    df['Lower'] = down
    return(df)

def STO(data, N=14, M=3):
    assert 'Low' in data.columns
    assert 'High' in data.columns
    assert 'Close' in data.columns
    
    data_ = pd.DataFrame()
    data_['low_N'] = data['Low'].rolling(N).min()
    data_['high_N'] = data['High'].rolling(N).max()
    data_['K'] = 100 * (data['Close'] - data_['low_N']) / \
        (data_['high_N'] - data_['low_N']) # The stochastic oscillator
    data_['D'] = data_['K'].rolling(M).mean() # the slow and smoothed K
    return data_

def ADX(high, low, close, lookback):
    
    df = pd.DataFrame()
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()
    df['+DI'] = plus_di
    df['-DI'] = minus_di
    df['ADX'] = adx_smooth
    
    return df

def MACD(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns = {'Close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    frames =  [macd, signal, hist]
    df = pd.concat(frames, join = 'inner', axis = 1)
    return df

def SuperTrend(high, low, close, lookback, multiplier):
    # ATR
    df = pd.DataFrame()
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.ewm(lookback).mean()
    
    # H/L AVG AND BASIC UPPER & LOWER BAND
    
    hl_avg = (high + low) / 2
    upper_band = (hl_avg + multiplier * atr).dropna()
    lower_band = (hl_avg - multiplier * atr).dropna()
    
    # FINAL UPPER BAND
    
    final_bands = pd.DataFrame(columns = ['upper', 'lower'])
    final_bands.iloc[:,0] = [x for x in upper_band - upper_band]
    final_bands.iloc[:,1] = final_bands.iloc[:,0]
    
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i,0] = 0
        else:
            if (upper_band[i] < final_bands.iloc[i-1,0]) | (close[i-1] > final_bands.iloc[i-1,0]):
                final_bands.iloc[i,0] = upper_band[i]
            else:
                final_bands.iloc[i,0] = final_bands.iloc[i-1,0]
    
    # FINAL LOWER BAND
    
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i, 1] = 0
        else:
            if (lower_band[i] > final_bands.iloc[i-1,1]) | (close[i-1] < final_bands.iloc[i-1,1]):
                final_bands.iloc[i,1] = lower_band[i]
            else:
                final_bands.iloc[i,1] = final_bands.iloc[i-1,1]
    
    # SUPERTREND
    
    supertrend = pd.DataFrame(columns = [f'supertrend_{lookback}'])
    supertrend.iloc[:,0] = [x for x in final_bands['upper'] - final_bands['upper']]
    
    for i in range(len(supertrend)):
        if i == 0:
            supertrend.iloc[i, 0] = 0
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] < final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] > final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] > final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] < final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
    
    supertrend = supertrend.set_index(upper_band.index)
    supertrend = supertrend.dropna()[1:]
    
    # ST UPTREND/DOWNTREND
    
    upt = []
    dt = []
    close = close.iloc[len(close) - len(supertrend):]

    for i in range(len(supertrend)):
        if close[i] > supertrend.iloc[i, 0]:
            upt.append(supertrend.iloc[i, 0])
            dt.append(np.nan)
        elif close[i] < supertrend.iloc[i, 0]:
            upt.append(np.nan)
            dt.append(supertrend.iloc[i, 0])
        else:
            upt.append(np.nan)
            dt.append(np.nan)
            
    st, upt, dt = pd.Series(supertrend.iloc[:, 0]), pd.Series(upt), pd.Series(dt)
    upt.index, dt.index = supertrend.index, supertrend.index
    
    df['ST'] = st
    df['UPT'] = upt
    df['DT'] = dt
    
    return df

def PRICE_plot(data, MA = False):
    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 3))
    #ax = plt.gca()
    ax1.set_facecolor('dimgrey')
    ax1.plot(data.index, data.Close, color = "deepskyblue")
    #ax1.set_xticks([])
    plt.grid()
    plt.show()


def STO_plot(data):
    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 1))
    ax1.set_facecolor('dimgrey')
    
    ax1.plot(data.index, data.D, color = "deepskyblue")
    ax1.plot(data.index, np.repeat(80,len(data)), color = "white", linestyle = "dotted")
    ax1.plot(data.index, np.repeat(20,len(data)), color = "white", linestyle = "dotted")
    plt.show()

def ADX_plot(data, components = False):    
    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 1))
    ax1.set_facecolor('dimgrey')
    
    if components == False:    
        ax1.plot(data.index, data['ADX'], color = "deepskyblue")
    
    elif components == True:
        ax1.plot(data.index, data['-DI'], color = "r")
        ax1.plot(data.index, data['+DI'], color = "g")
    
    ax1.plot(data.index, np.repeat(40,len(data)), color = "white", linestyle = "dotted")
    ax1.plot(data.index, np.repeat(20,len(data)), color = "white", linestyle = "dotted")
    plt.show()

def MACD_plot(prices, macd, signal, hist):
    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 1))
    ax1.set_facecolor('dimgrey')

    for i in range(len(prices)):
        if str(hist[i])[0] == '-':
            ax1.bar(prices.index[i], hist[i], color = 'gold')
        else:
            ax1.bar(prices.index[i], hist[i], color = 'deepskyblue')
    
    plt.grid()
    plt.show()
    
def TRADE_plot(data, buy_price, sell_price):
    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 5))
    ax1.set_facecolor('dimgrey')
    
    ax1.plot(data.index, data['Close'], linewidth = 2, color = "deepskyblue")
    ax1.plot(data.index, buy_price, marker = 'o', color = 'lime', markersize = 8, linewidth = 0)
    ax1.plot(data.index, sell_price, marker = 'X', color = 'tomato', markersize = 8, linewidth = 0)
    plt.grid()
    plt.show()

# not useful   
def RET_plot(data, strategy):
    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 5))
    ax1.set_facecolor('dimgrey')
    
    rets = data.Close.pct_change().dropna()
    strat_rets = strategy.position[1:]*rets

    ax1.plot(rets.index, rets, color = 'k', linewidth = 1)
    ax1.plot(strat_rets.index, strat_rets, color = 'deepskyblue', linewidth = 1)
    plt.grid()
    plt.show()
    
# very useful   
def SUM_plot(data, strategy):
    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 5))
    ax1.set_facecolor('dimgrey')
    
    rets = data.Close.pct_change().dropna()
    strat_rets = strategy.position[1:]*rets
    
    rets_cum = (1 + rets).cumprod() - 1 
    strat_cum = (1 + strat_rets).cumprod() - 1

    ax1.plot(rets_cum.index, rets_cum, color = 'w', linewidth = 2)
    ax1.plot(strat_cum.index, strat_cum, color = 'deepskyblue', linewidth = 2)
    plt.grid()
    plt.show()
    print(strat_cum[-1])
    


# PLOTTING THE TRADING SIGNALS

ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)
ax2 = plt.subplot2grid((8,1), (5,0), rowspan = 3, colspan = 1)

ax1.plot(aapl['Close'], color = 'skyblue', linewidth = 2, label = 'aapl')
ax1.plot(aapl.index, buy_price, marker = '^', color = 'green', markersize = 10, label = 'BUY SIGNAL', linewidth = 0)
ax1.plot(aapl.index, sell_price, marker = 'v', color = 'r', markersize = 10, label = 'SELL SIGNAL', linewidth = 0)
ax1.legend()
ax1.set_title('aapl MACD SIGNALS')

ax2.plot(aapl_macd['macd'], color = 'grey', linewidth = 1.5, label = 'MACD')
ax2.plot(aapl_macd['signal'], color = 'skyblue', linewidth = 1.5, label = 'SIGNAL')

for i in range(len(aapl_macd)):
    if str(aapl_macd['hist'][i])[0] == '-':
        ax2.bar(aapl_macd.index[i], aapl_macd['hist'][i], color = 'lime')
    else:
        ax2.bar(aapl_macd.index[i], aapl_macd['hist'][i], color = 'tomato')
        
plt.legend(loc = 'lower right')
plt.show()

def data(x, y, z):
    # x is a stock symbol
    # y is a start date
    # z is an end date
    # kan bruges på commodities, currencies og aktier
    
    #MSFT er en aktier der kan bruges
    #USDEUR=X er en currency der kan bruges
    #GC=F er guld priser
    
    st = pd.DataFrame()
    t = yf.Ticker(x)
    st = t.history(start=y, end=z)
    st.index = pd.to_datetime(st.index).tz_localize(None)
    return(st)

# Base data, Open, Low, High, Close, Volume, date points
DATA = data('BTC-USD','2020-01-01','2024-08-08')

#Stochastic oscillator of DATA
DATA_S = STO(DATA, N = 20, M = 3)

#ADX of DATA
DATA_AD = ADX(DATA['High'], DATA['Low'], DATA['Close'], 20)

#MACD of DATA
DATA_MACD = MACD(DATA['Close'], 26, 12, 9)

#SuperTrend of DATA
DATA_SUP = SuperTrend(DATA['High'], DATA['Low'], DATA['Close'], 20, 2)

#price curve
PRICE_plot(DATA)
# stochastic oscillator plot
STO_plot(DATA_S)
# Average directional index plot
ADX_plot(DATA_AD)

MACD_plot(DATA.Close, DATA_MACD['macd'], DATA_MACD['signal'], DATA_MACD['hist'])

TRADE_plot(DATA, buy_price, sell_price)

SUM_plot(DATA, strategy)

###############################################################################
#                        Portfolio level important functions
#                        VaR, CVaR, Drawdown, Annualized returns, etc.
#                        Not used for trading, but for profit considerations.
###############################################################################

def var(data, level=5):
    """
    """
    z = norm.ppf(level/100)
    return(-(data.mean() + z*data.std(ddof=0)))

def cvar(data, level=5):
    """
    """
    confidence_level = 1-(level/100)  # Set the desired confidence level
    sorted_prices = np.sort(data)
    num_samples = len(sorted_prices)
    cvar_index = int((1 - confidence_level) * num_samples)
    cvar = np.mean(sorted_prices[:cvar_index])
    return(cvar)

def drawdown(return_series):
    """
    """
    wealth_index = 1000*(1+return_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({"Drawdown": drawdowns})

def sharpe(self,x):
    """
    """
    mu = x.mean()
    sigma = np.sqrt(x.var())
    return((mu-5) / sigma)

#def sortino(risk_free,degree_of_freedom,growth_rate,minimum):
    #v=np.sqrt(np.abs(scipy.integrate.quad(lambda g ((risk_freeg)**2)*scipy.stats.t.pdf(g, degree_of_freedom), risk_free, minimum)))
    #s=(growth_rate-risk_free)/v[0]

    #return s
def UD_ind(data):
    """
    """
    result = pd.DataFrame()
    result['returns'] = data
    
    lst = ["","","",""]

    for i in range(4,len(data)):

        st = ""

        if data[i-3] < 0:
            st = "D"
        elif data[i-3] >= 0:
            st = "U"

        if data[i-2] < 0:
            st = st + "D"
        elif data[i-2] >= 0:
            st = st + "U"

        if data[i-1] < 0:
            st = st +"D"
        elif data[i-1] >= 0:
            st = st + "U"

        lst.append(st)

    result['UD3indicator'] = lst

    lst = ["", "", "", "", ""]

    for i in range(5,len(data)):

        st = ""

        if data.iloc[i-5] < 0:
            st = "D"
        elif data.iloc[i-5] >= 0:
            st = "U"

        if data.iloc[i-4] < 0:
            st = st + "D"
        elif data.iloc[i-4] >= 0:
            st = st + "U"

        if data.iloc[i-3] < 0:
            st = st +"D"
        elif data.iloc[i-3] >= 0:
            st = st + "U"

        if data.iloc[i-2] < 0:
            st = st +"D"
        elif data.iloc[i-2] >= 0:
            st = st +"U"

        if data.iloc[i-1] < 0:
            st = st + "D"
        elif data.iloc[i-1] >= 0:
            st = st + "U"

        lst.append(st)

    result['UD5indicator'] = lst
    
    return(result)

def trend_ind(data):
    """
    """
    assert data.columns[3] == "Close"
    assert data.columns[7] == "velocity"

    result = data.copy()
    Trend = [None] * len(result)
    for i in range(21, len(result)):
        Trend[i] = tm.mk_test(result['Close'][i-21:i], 'full', window = 21)[0]
    
    Trend2 = [None] * len(result)
    for i in range(21, len(result)):
        t_ind = np.mean(result['velocity'][i-21:i]) * 100
        if t_ind < -0.5:
            Trend2[i] = "decreasing"
        elif t_ind > 0.5:
            Trend2[i] = "increasing"
        else:
            Trend2[i] = "no trend"     
    result = pd.DataFrame([Trend, Trend2]).T
    return(result)

def hidden_states(data):
    data = data.dropna()
    data.replace([np.inf, -np.inf, np.nan], 1, inplace=True)

    close = np.array(data.iloc[1:,3])
    vel = np.array(data.iloc[1:,7])
    acc = np.array(data.iloc[1:,8])
    vol = np.array(data.iloc[1:,4])
    vol_vel = np.array(data.iloc[1:,9])
    vol_acc = np.array(data.iloc[1:,10])


    RSI = np.array(data.iloc[1:,11])
    MA3 = np.array(data.iloc[1:,13])
    STD3 = np.array(data.iloc[1:,14])
    MA5 = np.array(data.iloc[1:,18])
    STD5 = np.array(data.iloc[1:,19])
    MA10 = np.array(data.iloc[1:,23])
    STD10 = np.array(data.iloc[1:,24])
    MA60 = np.array(data.iloc[1:,38])
    STD60 = np.array(data.iloc[1:,39])


    X = np.column_stack([close, vel, acc, vol, vol_vel, vol_acc, RSI, MA3, STD3, MA5, STD5, MA10, STD10, MA60, STD60])
    model = GaussianHMM(n_components=6, covariance_type="diag", n_iter=10000, random_state = 123).fit(X)
    hidden_states = model.predict(X)
    data = data.iloc[1:,:]
    data['States'] = hidden_states
    return(data)

###############################################################################
#                        statistical and quantitative indicators
#                        Backtest, Up-down indicator, trend indicator etc.
#                        These need a series of updates
###############################################################################

# unsure of functionality
def backtest(x, window = 21):
    weights = x.shift(1).rolling(window= window).std().dropna() / np.mean(x)
    return(x * weights.shift(1)).sum()  

for i in range(len(prices)):
    if adx[i-1] < 25 and adx[i] > 25 and pdi[i] > ndi[i]:
        if signal != 1:
            buy_price.append(prices[i])
            sell_price.append(np.nan)
            signal = 1
            indicator.append(signal)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            indicator.append(0)
    elif adx[i-1] < 25 and adx[i] > 25 and ndi[i] > pdi[i]:
        if signal != -1:
            buy_price.append(np.nan)
            sell_price.append(prices[i])
            signal = -1
            indicator.append(signal)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            indicator.append(0)
    else:
        buy_price.append(np.nan)
        sell_price.append(np.nan)
        indicator.append(0)
        
for i in range(len(data)):
    if data['macd'][i] > data['signal'][i]:
        if signal != 1:
            buy_price.append(prices[i])
            sell_price.append(np.nan)
            signal = 1
            indicator.append(signal)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            indicator.append(0)
    elif data['macd'][i] < data['signal'][i]:
        if signal != -1:
            buy_price.append(np.nan)
            sell_price.append(prices[i])
            signal = -1
            indicator.append(signal)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            indicator.append(0)
    else:
        buy_price.append(np.nan)
        sell_price.append(np.nan)
        indicator.append(0)         
        

#def position_engine(data, strategy):

buy_price, sell_price, indicator = implement_macd_strategy(DATA['Close'], DATA_MACD)

position = []
for i in range(len(indicator)):
    if indicator[i] > 1:
        position.append(0)
    else:
        position.append(1)

for i in range(len(DATA['Close'])):
    if indicator[i] == 1:
        position[i] = 1
    elif indicator[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]
        
macd = DATA_MACD['macd']
indicator = DATA_MACD['signal']
close_price = DATA['Close']
signal = pd.DataFrame(indicator).rename(columns = {0:'signal'}).set_index(DATA.index)
position = pd.DataFrame(position).rename(columns = {0:'position'}).set_index(DATA.index)

frames = [close_price, macd, indicator, signal, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

close_price = DATA['Close']
plus_di = DATA_AD['+DI']
minus_di = DATA_AD['-DI']
indicator_ = DATA_AD['ADX']
signal = pd.DataFrame(indicator).rename(columns = {0:'signals'}).set_index(DATA.index)
position = pd.DataFrame(position).rename(columns = {0:'position'}).set_index(DATA.index)

frames = [close_price, plus_di, minus_di, indicator_, signal, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

val = 100
buy = False

for i in range(len(AAPL)):
    
    if signal == 1 and buy == False:
        buy = True
        val = val * 0.999
        buy_price = AAPL_S.Close[i]
    
    if signal == -1  and buy == True:
        val = (val + (AAPL_S.Close[i] - buy_price) ) * 0.999
        buy = False
        
val   